# **NLP Final Report**

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
!pip install -q pdfplumber pdf2image pytesseract
!apt-get -y install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import os
from pathlib import Path
import pandas as pd
import kagglehub

wattbot2025_path = kagglehub.competition_download("WattBot2025")
print("Downloaded to:", wattbot2025_path)

DATA_DIR  = Path(wattbot2025_path)
INPUT_DIR = DATA_DIR

WORK_DIR = Path("/content/wattbot_working")
WORK_DIR.mkdir(exist_ok=True, parents=True)

pdf_dir   = WORK_DIR / "pdfs"
text_dir  = WORK_DIR / "texts"
chunk_dir = WORK_DIR / "chunks"

for d in [pdf_dir, text_dir, chunk_dir]:
    d.mkdir(exist_ok=True, parents=True)

metadata = pd.read_csv(INPUT_DIR / "metadata.csv", encoding="latin-1")
train_qa = pd.read_csv(INPUT_DIR / "train_QA.csv", encoding="latin-1")
test_q   = pd.read_csv(INPUT_DIR / "test_Q.csv",  encoding="latin-1")

print(metadata.head())
print(train_qa.head())
print(test_q.head())


100%|██████████| 22.9k/22.9k [00:00<00:00, 25.7MB/s]

Extracting files...
Downloaded to: /root/.cache/kagglehub/competitions/WattBot2025
            id    type                                              title  \
0   amazon2023  report                  2023 Amazon Sustainability Report   
1     chen2024   paper  Efficient Heterogeneous Large Language Model D...   
2    chung2025   paper  The ML.ENERGY Benchmark: Toward Automated Infe...   
3  cottier2024   paper    The Rising Costs of Training Frontier AI Models   
4    dodge2022   paper  Measuring the Carbon Intensity of AI in Cloud ...   

   year                                           citation  \
0  2023  Amazon Staff. (2023). Amazon Sustainability Re...   
1  2024  Shaoyuan Chen, Wencong Xiao, Yutong Lin, Mingx...   
2  2025  Jae-Won Chung, Jiachen Liu, Jeff J. Ma, Ruofan...   
3  2024  Ben Cottier, Robi Rahman, Loredana Fattorini, ...   
4  2022  Jesse Dodge, Taylor Prewitt, Remi Tachet Des C...   

                                                 url  
0  https://sustainability.

In [ ]:
import os
import requests
import pdfplumber
from pdf2image import convert_from_path
import pytesseract

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 29.8 MB/s eta 0:00:00


In [ ]:
BASE_DIR = "/content"
pdf_dir = os.path.join(BASE_DIR, "papers")
text_dir = os.path.join(BASE_DIR, "processed_texts")

os.makedirs(pdf_dir, exist_ok=True)
os.makedirs(text_dir, exist_ok=True)

print("PDF 下載目錄:", pdf_dir)
print("文字輸出目錄:", text_dir)


PDF 下載目錄: /content/papers
文字輸出目錄: /content/processed_texts


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """使用 PyMuPDF 提取文字（保留空格）"""
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text("text") + "\n"
        doc.close()

        if not text.strip():
            print(f"⚠️ Using OCR for {pdf_path} ...")
            # OCR 備用方案
            from pdf2image import convert_from_path
            import pytesseract
            pages = convert_from_path(pdf_path, dpi=300)
            for page in pages:
                text += pytesseract.image_to_string(page, lang="eng") + "\n"

    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")

    return text.strip()

In [ ]:
def download_pdf(url, doc_id):
    pdf_path = os.path.join(pdf_dir, f"{doc_id}.pdf")
    if os.path.exists(pdf_path):
        return pdf_path

    try:
        print(f"Downloading {doc_id} ...")
        response = requests.get(url, timeout=30)
        if response.status_code == 200 and "pdf" in response.headers.get("Content-Type", ""):
            with open(pdf_path, "wb") as f:
                f.write(response.content)
            print(f"Saved {pdf_path}")
            return pdf_path
        else:
            print(f"⚠️ Failed {doc_id}: status {response.status_code}")
    except Exception as e:
        print(f"Error downloading {doc_id}: {e}")
    return None

In [ ]:
import os
import requests
import warnings
import logging
from pdfminer.high_level import extract_text

# 抑制警告
logging.getLogger('pdfminer').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

def download_pdf(url, doc_id, papers_dir="papers"):
    """下載 PDF 文件"""
    os.makedirs(papers_dir, exist_ok=True)
    pdf_path = os.path.join(papers_dir, f"{doc_id}.pdf")

    if os.path.exists(pdf_path):
        return pdf_path

    print(f"⬇️ Downloading {doc_id} ...")
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code == 200 and response.content[:4] == b'%PDF':
            with open(pdf_path, "wb") as f:
                f.write(response.content)
            print(f"✅ Saved {pdf_path}")
            return pdf_path
        else:
            print(f"⚠️ Failed {doc_id}: status {response.status_code}")
    except Exception as e:
        print(f"⚠️ Failed {doc_id}: {type(e).__name__}")
    return None

def extract_text_from_pdf(pdf_path):
    """提取 PDF 文字"""
    try:
        return extract_text(pdf_path)
    except:
        return ""

# ===== 步驟 1: 修正 zschache2025 的 URL =====
print("🔧 更新 zschache2025 的 URL...")
metadata.loc[metadata['id'] == 'zschache2025', 'url'] = 'https://arxiv.org/pdf/2508.14170'
print("✅ URL 已更新\n")

# ===== 步驟 2: 主循環處理所有文件 =====
text_dir = "processed_texts"
os.makedirs(text_dir, exist_ok=True)

success_count = 0
fail_count = 0
failed_ids = []

for i, row in metadata.iterrows():
    doc_id = row["id"]
    url = row["url"]

    # 下載 PDF
    pdf_path = download_pdf(url, doc_id)

    # 如果下載成功,提取文字
    if pdf_path:
        text = extract_text_from_pdf(pdf_path)
        if text.strip():
            out_path = os.path.join(text_dir, f"{doc_id}.txt")
            with open(out_path, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"📝 Saved text -> {out_path}\n")
            success_count += 1
        else:
            print(f"⚠️ {doc_id}: No text extracted\n")
            fail_count += 1
            failed_ids.append(doc_id)
    else:
        fail_count += 1
        failed_ids.append(doc_id)

# ===== 步驟 3: 顯示最終統計結果 =====
print("="*70)
print("🎉 所有處理完成!")
print("="*70)
print(f"✅ 成功: {success_count}/{len(metadata)} 篇")
print(f"❌ 失敗: {fail_count}/{len(metadata)} 篇")
print(f"📊 成功率: {success_count/len(metadata)*100:.1f}%")

if failed_ids:
    print(f"\n失敗的文件 ID:")
    for doc_id in failed_ids:
        print(f"  • {doc_id}")
else:
    print(f"\n🌟 完美!所有 {len(metadata)} 篇論文都已成功處理!")

print("="*70)

# ===== 步驟 4: 驗證文字檔數量 =====
text_files = [f for f in os.listdir(text_dir) if f.endswith('.txt')]
print(f"\n📁 已產生的文字檔數量: {len(text_files)}")

# 特別驗證 zschache2025 是否成功
if "zschache2025.txt" in text_files:
    print(f"✅ zschache2025.txt 已成功產生!")
else:
    print(f"⚠️ zschache2025.txt 尚未產生")

🔧 更新 zschache2025 的 URL...
✅ URL 已更新

⬇️ Downloading amazon2023 ...
✅ Saved papers/amazon2023.pdf
📝 Saved text -> processed_texts/amazon2023.txt

⬇️ Downloading chen2024 ...
✅ Saved papers/chen2024.pdf
📝 Saved text -> processed_texts/chen2024.txt

⬇️ Downloading chung2025 ...
✅ Saved papers/chung2025.pdf
📝 Saved text -> processed_texts/chung2025.txt

⬇️ Downloading cottier2024 ...
✅ Saved papers/cottier2024.pdf
📝 Saved text -> processed_texts/cottier2024.txt

⬇️ Downloading dodge2022 ...
✅ Saved papers/dodge2022.pdf
📝 Saved text -> processed_texts/dodge2022.txt

⬇️ Downloading ebert2024 ...
✅ Saved papers/ebert2024.pdf
📝 Saved text -> processed_texts/ebert2024.txt

⬇️ Downloading erben2023 ...
✅ Saved papers/erben2023.pdf
📝 Saved text -> processed_texts/erben2023.txt

⬇️ Downloading fernandez2025 ...
✅ Saved papers/fernandez2025.pdf
📝 Saved text -> processed_texts/fernandez2025.txt

⬇️ Downloading griggs2024 ...
✅ Saved papers/griggs2024.pdf
📝 Saved text -> processed_texts/griggs2024.t

### **1. Corpus Side**

### **1.1 Environment Setup**

In [ ]:
import subprocess, sys

packages = [
    "pdfplumber",
    "sentence-transformers",
    "faiss-cpu",
    "tqdm",
    "fuzzywuzzy",
    "python-Levenshtein",
    "rank-bm25",
]

for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])


### **1.2 Module Import**

In [ ]:
import os
import re
import json
import time
import ast
from pathlib import Path
from typing import List, Dict, Optional

import numpy as np
import pandas as pd
import requests
import pdfplumber
import faiss

from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
from rank_bm25 import BM25Okapi


### **1.3 Dataset Loading**






In [ ]:
INPUT_DIR = Path("/root/.cache/kagglehub/competitions/WattBot2025")

WORK_DIR = Path("/content/wattbot_working")
WORK_DIR.mkdir(exist_ok=True, parents=True)

pdf_dir   = WORK_DIR / "pdfs"
text_dir  = WORK_DIR / "texts"
chunk_dir = WORK_DIR / "chunks"

for d in [pdf_dir, text_dir, chunk_dir]:
    d.mkdir(exist_ok=True, parents=True)

metadata = pd.read_csv(INPUT_DIR / "metadata.csv", encoding="latin1")
train_qa = pd.read_csv(INPUT_DIR / "train_QA.csv", encoding="latin1")
test_q   = pd.read_csv(INPUT_DIR / "test_Q.csv",  encoding="latin1")

print("metadata:", metadata.shape)
print("train_qa:", train_qa.shape)
print("test_q:",   test_q.shape)

metadata: (32, 6)
train_qa: (41, 9)
test_q: (282, 9)


### **1.4 Text Cleaning**

In [ ]:
"""
完整處理流程 - 所有 32 篇文檔
"""

import re
import unicodedata
import json
from pathlib import Path
from collections import defaultdict

# 確認路徑
text_dir = Path("/content/processed_texts")
chunk_dir = Path("/content/wattbot_working/chunks")
chunk_dir.mkdir(exist_ok=True, parents=True)

def split_long_paragraph(text: str, max_words: int = 300) -> list:
    """將超長段落按句子切分"""
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = []
    current_words = 0

    for sent in sentences:
        sent_words = len(sent.split())
        if current_words + sent_words > max_words and current:
            chunks.append(' '.join(current))
            current = [sent]
            current_words = sent_words
        else:
            current.append(sent)
            current_words += sent_words

    if current:
        chunks.append(' '.join(current))
    return chunks

def clean_and_chunk_document(text_file: Path, doc_id: str, doc_type: str, chunk_words: int = 200):
    """處理單一文檔"""
    try:
        with open(text_file, 'r', encoding='utf-8') as f:
            text = f.read()
    except:
        return []

    if not text.strip():
        return []

    text = unicodedata.normalize("NFKC", text)
    text = text.replace("ﬁ", "fi").replace("ﬂ", "fl")
    text = text.replace("\r\n", "\n").replace("\r", "\n")

    if doc_type == "paper":
        ref_patterns = [
            r"\n\s*References\s*\n",
            r"\n\s*REFERENCES\s*\n",
            r"\n\s*Bibliography\s*\n",
            r"\n\s*BIBLIOGRAPHY\s*\n",
            r"\nReferences\s*\[",
        ]
        for pattern in ref_patterns:
            match = re.search(pattern, text, flags=re.IGNORECASE)
            if match:
                text = text[:match.start()]
                print(f"✂️ Removed references from {doc_id}")
                break

    text = re.sub(r"(\w)-\s*\n\s*(\w)", r"\1\2", text)
    text = re.sub(r"(?<!\n)(?<=[a-zA-Z0-9])\n(?=[a-zA-Z0-9])", " ", text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r',([A-Z])', r', \1', text)
    text = re.sub(r'([a-z0-9])\.([A-Z])', r'\1. \2', text)
    text = re.sub(r'(\])([A-Z])', r'\1 \2', text)
    text = re.sub(r'([A-Z][a-z]+)(\d)', r'\1 \2', text)
    text = re.sub(r'(\d)([A-Z][a-z])', r'\1 \2', text)

    text = re.sub(r"(?m)^\s*\d+\s*$", "", text)
    text = re.sub(r"(?m)^\s*Page\s+\d+.*$", "", text, flags=re.IGNORECASE)

    if doc_type == "report":
        text = re.sub(r"(?mi)^.*?(sustainability|amazon\.com).*$", "", text)

    paragraphs = text.split("\n\n")
    cleaned_pars = []
    noise_kw = ["creative commons", "cc by", "arxiv preprint", "copyright",
                "all rights reserved", "licensed", "preprint", "arxiv:"]

    for p in paragraphs:
        p = p.strip()
        if not p or len(p.split()) < 5:
            continue
        if any(kw in p.lower() for kw in noise_kw):
            continue
        cleaned_pars.append(p)

    if not cleaned_pars:
        return []

    text = "\n\n".join(cleaned_pars)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = text.strip()

    paragraphs = [p for p in text.split("\n\n") if p.strip()]
    chunks = []
    current = []
    current_words = 0

    for para in paragraphs:
        words = len(para.split())

        if words > chunk_words * 1.3:
            if current:
                chunks.append("\n\n".join(current))
                current = []
                current_words = 0
            sub_chunks = split_long_paragraph(para, max_words=chunk_words)
            chunks.extend(sub_chunks)
            continue

        if current_words + words > chunk_words and current_words >= 80:
            chunks.append("\n\n".join(current))
            current = [para]
            current_words = words
        else:
            current.append(para)
            current_words += words

    if current:
        chunks.append("\n\n".join(current))

    result = []
    for i, chunk_text in enumerate(chunks):
        if len(chunk_text.split()) >= 30:
            result.append({
                "doc_id": doc_id,
                "chunk_id": i,
                "type": "text",
                "content": chunk_text,
                "word_count": len(chunk_text.split())
            })

    return result

# 執行完整處理
print("🚀 開始處理全部 32 篇文檔...\n")

all_chunks = []
stats = {"success": 0, "failed": []}

for _, row in metadata.iterrows():
    doc_id = row['id']
    doc_type = row['type']
    text_file = text_dir / f"{doc_id}.txt"

    if not text_file.exists():
        print(f"⚠️  {doc_id}.txt 不存在")
        stats["failed"].append(doc_id)
        continue

    try:
        chunks = clean_and_chunk_document(text_file, doc_id, doc_type, chunk_words=200)

        if chunks:
            all_chunks.extend(chunks)
            stats["success"] += 1
            avg = sum(c['word_count'] for c in chunks) / len(chunks)
            print(f"✓ {doc_id:20s} | {len(chunks):3d} chunks | avg {avg:.0f} words")
        else:
            print(f"⚠️  {doc_id} 無內容")
            stats["failed"].append(doc_id)

    except Exception as e:
        print(f"✗ {doc_id} 失敗: {e}")
        stats["failed"].append(doc_id)

# 儲存
output_file = chunk_dir / "chunks_all.jsonl"
with open(output_file, 'w', encoding='utf-8') as f:
    for chunk in all_chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + '\n')

# 統計
print("\n" + "="*70)
print(f"✅ 完成！成功: {stats['success']}/32")
print(f"📊 總 chunks: {len(all_chunks)}")
if all_chunks:
    avg = sum(c['word_count'] for c in all_chunks) / len(all_chunks)
    print(f"📏 平均大小: {avg:.1f} words")
if stats['failed']:
    print(f"❌ 失敗: {stats['failed']}")
print(f"💾 輸出: {output_file}")
print("="*70)

if all_chunks:
    word_counts = [c['word_count'] for c in all_chunks]
    print(f"\n📊 字數分布:")
    print(f"  最小: {min(word_counts)}")
    print(f"  最大: {max(word_counts)}")
    print(f"  中位數: {sorted(word_counts)[len(word_counts)//2]}")

    doc_chunks = defaultdict(int)
    for c in all_chunks:
        doc_chunks[c['doc_id']] += 1

    print(f"\n📄 每篇文檔的 chunks:")
    print(f"  平均: {sum(doc_chunks.values())/len(doc_chunks):.1f}")
    print(f"  範圍: {min(doc_chunks.values())} - {max(doc_chunks.values())}")

print("\n✨ 準備建立檢索系統！")

🚀 開始處理全部 32 篇文檔...

✓ amazon2023           | 363 chunks | avg 167 words
✂️ Removed references from chen2024
✓ chen2024             |  48 chunks | avg 161 words
✂️ Removed references from chung2025
✓ chung2025            |  31 chunks | avg 173 words
✂️ Removed references from cottier2024
✓ cottier2024          |  33 chunks | avg 161 words
✂️ Removed references from dodge2022
✓ dodge2022            |  47 chunks | avg 184 words
✂️ Removed references from ebert2024
✓ ebert2024            |  63 chunks | avg 154 words
✓ erben2023            |  84 chunks | avg 160 words
✂️ Removed references from fernandez2025
✓ fernandez2025        |  34 chunks | avg 159 words
✂️ Removed references from griggs2024
✓ griggs2024           |  37 chunks | avg 159 words
✂️ Removed references from han2024
✓ han2024              |  76 chunks | avg 159 words
✂️ Removed references from jegham2025
✓ jegham2025           |  43 chunks | avg 155 words
✂️ Removed references from khan2025
✓ khan2025             |  22 chunk

### **1.6 Document Parsing**

In [ ]:
!pip install -q pymupdf rank-bm25 sentence-transformers faiss-cpu chardet

In [ ]:
"""
Part 1: 測試 PDF Parsing 能否找到關鍵資訊
檢查表格、數字是否能正確提取
"""

import fitz  # PyMuPDF
from pathlib import Path
import pandas as pd

# 讀取資料
data_dir = Path("/root/.cache/kagglehub/competitions/WattBot2025")
train_qa = pd.read_csv(data_dir / 'train_QA.csv', encoding='utf-8-sig')
pdf_dir = Path("/content/papers")

# 測試案例
test_cases = [
    {
        'id': 'q054',
        'ref_id': 'chen2024',
        'answer': '64.7 GB',
        'keywords': ['LLaMA-33B', '64.7', 'Table 3'],
        'question': train_qa[train_qa['id'] == 'q054']['question'].values[0]
    },
    {
        'id': 'q297',
        'ref_id': 'zschache2025',
        'answer': '8.720430108',
        'keywords': ['Qwen', '7B', '72B', '5.58', '48.66'],
        'question': train_qa[train_qa['id'] == 'q297']['question'].values[0]
    },
    {
        'id': 'q009',
        'ref_id': 'patterson2021',
        'answer': '4.3 tCO2e',
        'keywords': ['GShard-600B', '4.3', 'tCO2e'],
        'question': train_qa[train_qa['id'] == 'q009']['question'].values[0]
    },
]

print("=" * 80)
print("🔍 測試：PDF Parsing 能否找到答案的關鍵資訊")
print("=" * 80)

overall_results = []

for test in test_cases:
    print(f"\n{'='*80}")
    print(f"📋 題目 {test['id']}: {test['question'][:100]}...")
    print(f"✓ 期望答案: {test['answer']}")
    print(f"📄 文件: {test['ref_id']}")
    print("-" * 80)

    pdf_path = pdf_dir / f"{test['ref_id']}.pdf"

    if not pdf_path.exists():
        print(f"❌ PDF 不存在")
        overall_results.append({'id': test['id'], 'found_ratio': 0})
        continue

    # 開啟 PDF
    doc = fitz.open(pdf_path)

    # 搜尋每個關鍵字
    found_info = {}

    for kw in test['keywords']:
        found = False

        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()

            if kw in text:
                found = True
                idx = text.find(kw)
                # 取前後 150 字元作為上下文
                context = text[max(0, idx-150):min(len(text), idx+150)]
                found_info[kw] = {
                    'page': page_num + 1,
                    'context': context.replace('\n', ' ').strip()
                }
                break

        if found:
            print(f"  ✓ 找到 '{kw}' 在第 {found_info[kw]['page']} 頁")
            print(f"    上下文: ...{found_info[kw]['context'][:100]}...")
        else:
            print(f"  ✗ 未找到 '{kw}'")

    doc.close()

    # 判斷結果
    found_count = len(found_info)
    total_count = len(test['keywords'])
    ratio = found_count / total_count

    overall_results.append({
        'id': test['id'],
        'found_ratio': ratio,
        'found': found_count,
        'total': total_count
    })

    if found_count == total_count:
        print(f"\n  ✅ 所有關鍵字都找到了！({found_count}/{total_count})")
    else:
        print(f"\n  ⚠️  只找到部分關鍵字 ({found_count}/{total_count})")

# 總結
print("\n" + "=" * 80)
print("📊 診斷結果總結")
print("=" * 80)

avg_ratio = sum(r['found_ratio'] for r in overall_results) / len(overall_results)
print(f"\n平均找到率: {avg_ratio:.1%}")

for r in overall_results:
    print(f"  {r['id']}: {r['found']}/{r['total']} ({r['found_ratio']:.0%})")

print("\n💡 建議:")
if avg_ratio >= 0.8:
    print("  ✅ Parsing 效果良好，可以繼續使用當前的提取方法")
elif avg_ratio >= 0.5:
    print("  ⚠️  Parsing 效果一般，建議:")
    print("     1. 檢查表格提取（pdfplumber 對表格效果更好）")
    print("     2. 加強數字提取")
else:
    print("  ❌ Parsing 效果不佳，建議:")
    print("     1. 使用 pdfplumber 處理表格")
    print("     2. 考慮 OCR（如果內容是圖片）")

print("=" * 80)

🔍 測試：PDF Parsing 能否找到答案的關鍵資訊

📋 題目 q054: What is the model size in gigabytes (GB) for the LLaMA-33B model?...
✓ 期望答案: 64.7 GB
📄 文件: chen2024
--------------------------------------------------------------------------------
  ✓ 找到 'LLaMA-33B' 在第 9 頁
    上下文: ...similar outlines and workload charac- teristics and only have minor differences irrelevant to system...
  ✓ 找到 '64.7' 在第 9 頁
    上下文: ...the same sequence length for evaluation. The summaries of these Table 3: Large language models used ...
  ✓ 找到 'Table 3' 在第 9 頁
    上下文: ...architectures have similar outlines and workload charac- teristics and only have minor differences i...

  ✅ 所有關鍵字都找到了！(3/3)

📋 題目 q297: When comparing small and large versions of Qwen models in zero-shot classification, how many times m...
✓ 期望答案: 8.720430108
📄 文件: zschache2025
--------------------------------------------------------------------------------
  ✓ 找到 'Qwen' 在第 5 頁
    上下文: ...Llama 3.1 8B https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Inst

### **1.7 Smart Chunking & 結構化提取PDF檔案**

In [ ]:
from typing import List, Dict
from pathlib import Path
import json
import re
from tqdm import tqdm
import pdfplumber
import fitz
import unicodedata

pdf_dir = Path("/content/papers")
chunk_dir = Path("/content/chunks")
chunk_dir.mkdir(exist_ok=True, parents=True)

print("📂 Step 1: 結構化提取 PDF...")

def clean_text(text: str) -> str:
    if not text:
        return ""

    text = unicodedata.normalize("NFKC", text)
    text = text.replace("ﬁ", "fi").replace("ﬂ", "fl")
    text = text.replace("\r\n", "\n").replace("\r", "\n")

    text = re.sub(r"(\w)-\s*\n\s*(\w)", r"\1\2", text)
    text = re.sub(r"(?<!\n)(?<=[a-zA-Z0-9])\n(?=[a-zA-Z0-9])", " ", text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r',([A-Z])', r', \1', text)
    text = re.sub(r'([a-z0-9])\.([A-Z])', r'\1. \2', text)
    text = re.sub(r'(\])([A-Z])', r'\1 \2', text)
    text = re.sub(r'([A-Z][a-z]+)(\d)', r'\1 \2', text)
    text = re.sub(r'(\d)([A-Z][a-z])', r'\1 \2', text)
    text = re.sub(r'(\d)([a-z]+)', r'\1 \2', text)

    text = re.sub(r'\s+', ' ', text)
    return text.strip()

class StructuredDocumentExtractor:
    def extract(self, pdf_path: Path) -> Dict:
        doc_id = pdf_path.stem
        result = {
            "doc_id": doc_id,
            "pages": [],
            "tables": [],
            "figures": [],
            "full_text": "",
        }

        try:
            doc = fitz.open(pdf_path)
            page_texts = []

            for page_num in range(len(doc)):
                page = doc[page_num]
                page_idx = page_num + 1

                raw_text = page.get_text("text")
                clean = clean_text(raw_text) if raw_text else ""

                if clean:
                    page_texts.append(clean)

                    if re.search(r"Figure\s+\d+", clean, re.IGNORECASE):
                        captions = re.findall(
                            r"(Figure\s+\d+[:\.].*?)(?=\n\n|\n[A-Z]|$)",
                            clean, re.IGNORECASE | re.DOTALL
                        )
                        for cap in captions:
                            result["figures"].append({"page": page_idx, "caption": cap.strip()})

            doc.close()

            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    page_idx = page_num + 1
                    tables = page.extract_tables()

                    for table_idx, table in enumerate(tables or []):
                        if table and len(table) > 1:
                            headers = [str(cell).strip() if cell else "" for cell in table[0]]
                            rows = []
                            for row in table[1:]:
                                clean_row = [str(cell).strip() if cell else "" for cell in row]
                                if any(clean_row):
                                    rows.append(clean_row)

                            if rows and any(headers):
                                table_id = f"Table_{page_idx}_{table_idx+1}"
                                structured_table = {
                                    "id": table_id,
                                    "page": page_idx,
                                    "headers": headers,
                                    "rows": rows,
                                }
                                result["tables"].append(structured_table)

            if page_texts:
                result["full_text"] = "\n\n".join(page_texts)

        except Exception as e:
            print(f"✗ {doc_id}: {e}")

        return result

extractor = StructuredDocumentExtractor()
all_docs = {}

for pdf_file in tqdm(list(pdf_dir.glob("*.pdf")), desc="提取 PDF"):
    doc_data = extractor.extract(pdf_file)
    if doc_data["full_text"]:
        all_docs[doc_data["doc_id"]] = doc_data

print(f"\n✅ 提取完成: {len(all_docs)} 文檔, {sum(len(d['tables']) for d in all_docs.values())} 表格\n")

class HybridChunker:
    def __init__(self, text_chunk_words: int = 200):
        self.text_chunk_words = text_chunk_words

    def chunk_document(self, doc_id: str, doc_data: Dict) -> List[Dict]:
        chunks = []

        full_text = doc_data.get("full_text", "")
        if full_text:
            text_chunks = self._chunk_text(full_text)
            for text in text_chunks:
                chunks.append({
                    "doc_id": doc_id,
                    "chunk_id": len(chunks),
                    "type": "text",
                    "content": text,
                    "word_count": len(text.split()),
                })

        for table in doc_data.get("tables", []):
            table_content = self._format_table(table)
            if (table_content and len(table_content.split()) > 10 and any(h for h in table.get('headers', []) if h.strip())): #原本是15
                chunks.append({
                    "doc_id": doc_id,
                    "chunk_id": len(chunks),
                    "type": "table",
                    "content": table_content,
                    "word_count": len(table_content.split()),
                })

        for fig in doc_data.get("figures", []):
            caption = fig.get("caption", "").strip()
            if caption and len(caption.split()) >= 5:
                chunks.append({
                    "doc_id": doc_id,
                    "chunk_id": len(chunks),
                    "type": "figure",
                    "content": f"Figure: {caption}",
                    "word_count": len(caption.split()),
                })

        return chunks

    def _chunk_text(self, text: str) -> List[str]:
        paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
        chunks = []
        current = []
        current_words = 0

        for para in paragraphs:
            words = len(para.split())

            if words > self.text_chunk_words * 1.3:
                if current:
                    chunks.append("\n\n".join(current))
                    current = []
                    current_words = 0

                sentences = re.split(r'(?<=[.!?])\s+', para)
                temp = []
                temp_words = 0
                for sent in sentences:
                    sent_words = len(sent.split())
                    if temp_words + sent_words > self.text_chunk_words and temp:
                        chunks.append(' '.join(temp))
                        temp = [sent]
                        temp_words = sent_words
                    else:
                        temp.append(sent)
                        temp_words += sent_words
                if temp:
                    chunks.append(' '.join(temp))
                continue

            if current_words + words > self.text_chunk_words and current_words >= 80:
                chunks.append("\n\n".join(current))
                current = [para]
                current_words = words
            else:
                current.append(para)
                current_words += words

        if current:
            chunks.append("\n\n".join(current))

        return [c for c in chunks if len(c.split()) >= 20] #原本是30! 改更寬鬆一點

    def _format_table(self, table: Dict) -> str:
        lines = [f"[{table['id']} on Page {table['page']}]"]

        headers = table.get('headers', [])
        rows = table.get('rows', [])

        if headers and rows:
            clean_headers = [h.strip() for h in headers if h.strip()]

            summary_parts = []
            for row in rows[:3]:
                clean_row = [cell.strip() for cell in row if cell.strip()]
                if len(clean_row) >= 2:
                    if len(clean_headers) == len(clean_row):
                        row_desc = []
                        for header, value in zip(clean_headers, clean_row):
                            row_desc.append(f"{value} ({header})")
                        summary_parts.append(", ".join(row_desc))

            if summary_parts:
                lines.append("Summary: " + "; ".join(summary_parts[:2]))

        if headers and any(headers):
            lines.append("\nStructured Data:")
            clean_headers = [h.strip() for h in headers if h.strip()]
            lines.append(" | ".join(clean_headers))

        if rows:
            for row in rows:
                if any(row):
                    clean_row = [cell.strip() for cell in row if cell.strip()]
                    if clean_row:
                        lines.append(" | ".join(clean_row))

        return "\n".join(lines)

chunker = HybridChunker(text_chunk_words=200)
all_chunks = []

for doc_id, doc_data in tqdm(all_docs.items(), desc="分塊"):
    chunks = chunker.chunk_document(doc_id, doc_data)
    all_chunks.extend(chunks)

output_file = Path("/content/hybrid_chunks.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)

from collections import Counter
type_counts = Counter(c['type'] for c in all_chunks)
print(f"\n✅ 完成: {len(all_chunks)} chunks")
for t, c in type_counts.items():
    print(f"  {t}: {c}")

📂 Step 1: 結構化提取 PDF...


提取 PDF: 100%|██████████| 32/32 [02:43<00:00,  5.11s/it]



✅ 提取完成: 32 文檔, 145 表格



分塊: 100%|██████████| 32/32 [00:00<00:00, 172.12it/s]



✅ 完成: 2608 chunks
  text: 2372
  table: 134
  figure: 102


### **1.8 BM25重點測試**

In [ ]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
from rank_bm25 import BM25Okapi
from typing import List, Dict
import re

print("📂 載入 chunks...\n")

text_chunks_file = Path("/content/wattbot_working/chunks/chunks_all.jsonl")
text_chunks = []
with open(text_chunks_file, 'r', encoding='utf-8') as f:
    for line in f:
        text_chunks.append(json.loads(line))
print(f"✓ 純文字 chunks: {len(text_chunks)}")

hybrid_chunks_file = Path("/content/hybrid_chunks.json")
with open(hybrid_chunks_file, 'r', encoding='utf-8') as f:
    hybrid_chunks = json.load(f)
print(f"✓ 混合 chunks: {len(hybrid_chunks)}")

print("\n🔨 建立雙重 BM25 索引...\n")

def tokenize(text: str) -> List[str]:
    text = text.lower()
    tokens = re.findall(r'\d+\.?\d*%?|\w+', text)
    return tokens

text_corpus = [c['content'] for c in text_chunks]
text_tokenized = [tokenize(t) for t in text_corpus]
bm25_text = BM25Okapi(text_tokenized)
print("✓ 純文字 BM25 索引完成")

table_chunks = [c for c in hybrid_chunks if c['type'] == 'table']
table_corpus = [c['content'] for c in table_chunks]
table_tokenized = [tokenize(t) for t in table_corpus]
bm25_table = BM25Okapi(table_tokenized)
print(f"✓ 表格 BM25 索引完成 ({len(table_chunks)} 個表格)\n")

def is_table_query(query: str) -> bool:
    table_keywords = [
        'table', 'how many', 'how much', 'number', 'size', 'gigabytes',
        'gb', 'mb', 'tb', 'watts', 'kwh', 'carbon', 'co2', 'emissions',
        'cost', 'dollar', 'percentage', '%', 'comparing', 'compare',
        'versus', 'vs', 'difference between', 'times more', 'times less'
    ]
    query_lower = query.lower()
    return any(kw in query_lower for kw in table_keywords)

def hybrid_search(query: str, top_k: int = 10) -> List[Dict]:
    tokenized_query = tokenize(query)
    text_scores = bm25_text.get_scores(tokenized_query)

    if is_table_query(query):
        table_scores = bm25_table.get_scores(tokenized_query)
        text_top_indices = np.argsort(text_scores)[-7:][::-1]
        table_top_indices = np.argsort(table_scores)[-3:][::-1]

        results = []

        for idx in text_top_indices:
            results.append({
                'rank': len(results) + 1,
                'score': float(text_scores[idx]),
                'doc_id': text_chunks[idx]['doc_id'],
                'type': 'text',
                'content': text_chunks[idx]['content'],
                'source': 'text_index'
            })

        for idx in table_top_indices:
            results.append({
                'rank': len(results) + 1,
                'score': float(table_scores[idx]) * 0.8,
                'doc_id': table_chunks[idx]['doc_id'],
                'type': 'table',
                'content': table_chunks[idx]['content'],
                'source': 'table_index'
            })

        results.sort(key=lambda x: x['score'], reverse=True)

        for i, r in enumerate(results):
            r['rank'] = i + 1

        return results[:top_k]

    else:
        top_indices = np.argsort(text_scores)[-top_k:][::-1]
        results = []
        for rank, idx in enumerate(top_indices, 1):
            results.append({
                'rank': rank,
                'score': float(text_scores[idx]),
                'doc_id': text_chunks[idx]['doc_id'],
                'type': 'text',
                'content': text_chunks[idx]['content'],
                'source': 'text_index'
            })
        return results

def get_doc_from_chunks(chunks_results: List[Dict]) -> List[str]:
    seen = set()
    docs = []
    for r in chunks_results:
        doc_id = r['doc_id']
        if doc_id not in seen:
            seen.add(doc_id)
            docs.append(doc_id)
    return docs

print("📊 評估混合檢索策略...\n")

data_dir = Path("/root/.cache/kagglehub/competitions/WattBot2025")
train_qa = pd.read_csv(data_dir / 'train_QA.csv', encoding='utf-8-sig')

def evaluate_hybrid(qa_df: pd.DataFrame):
    results = {k: [] for k in [1, 3, 5, 10]}
    mrr_scores = []
    valid_count = 0
    table_query_count = 0

    for idx, row in qa_df.iterrows():
        question = row['question']
        ref_id_raw = row['ref_id']

        if pd.isna(ref_id_raw) or str(ref_id_raw).strip() == '':
            continue

        try:
            if isinstance(ref_id_raw, str):
                if ref_id_raw.startswith('['):
                    import ast
                    true_docs = ast.literal_eval(ref_id_raw)
                    if not isinstance(true_docs, list):
                        true_docs = [true_docs]
                else:
                    true_docs = [ref_id_raw.strip()]
            else:
                true_docs = [str(ref_id_raw).strip()]
        except:
            continue

        valid_count += 1

        if is_table_query(question):
            table_query_count += 1

        search_results = hybrid_search(question, top_k=10)
        retrieved_docs = get_doc_from_chunks(search_results)

        for k in [1, 3, 5, 10]:
            top_k_docs = retrieved_docs[:k]
            hit = any(doc in top_k_docs for doc in true_docs)
            results[k].append(1 if hit else 0)

        reciprocal_rank = 0
        for rank, doc_id in enumerate(retrieved_docs, 1):
            if doc_id in true_docs:
                reciprocal_rank = 1.0 / rank
                break
        mrr_scores.append(reciprocal_rank)

        if valid_count % 10 == 0:
            print(f"  處理: {valid_count} 問題...", end='\r')

    print(f"  ✓ 完成 {valid_count} 問題" + " "*20)

    metrics = {}
    for k in [1, 3, 5, 10]:
        metrics[f'Recall@{k}'] = np.mean(results[k])
    metrics['MRR'] = np.mean(mrr_scores)
    metrics['total_questions'] = valid_count
    metrics['table_queries'] = table_query_count

    return metrics

metrics = evaluate_hybrid(train_qa)

print("\n" + "="*70)
print("📈 混合檢索效果（智能路由）")
print("="*70)
print(f"評估問題數: {metrics['total_questions']}")
print(f"表格查詢數: {metrics['table_queries']} ({metrics['table_queries']/metrics['total_questions']*100:.1f}%)")
print()
for k in [1, 3, 5, 10]:
    recall = metrics[f'Recall@{k}']
    bar = int(recall * 50)
    print(f"  Recall@{k:2d}  : {recall:6.2%}  {'█' * bar}")

print(f"\n  MRR        : {metrics['MRR']:.4f}")
print("="*70)

print("\n🔍 測試關鍵案例...\n")

test_cases = [
    ('q054', 'chen2024', 'What is the model size in gigabytes (GB) for the LLaMA-33B model?'),
    ('q297', 'zschache2025', 'When comparing small and large versions of Qwen models in zero-shot classification, how many times more energy does the large model consume?'),
    ('q009', 'patterson2021', 'What were the net CO2e emissions from training the GShard-600B model?'),
]

for qid, expected_doc, question in test_cases:
    print(f"{'─'*70}")
    print(f"📋 {qid}: {question[:80]}...")
    print(f"✓ 期望: {expected_doc}")

    is_table = is_table_query(question)
    print(f"🔍 查詢類型: {'表格查詢 📊' if is_table else '文字查詢 📄'}\n")

    results = hybrid_search(question, top_k=5)

    print("檢索結果:")
    for r in results:
        marker = "✅" if r['doc_id'] == expected_doc else "  "
        icon = "📊" if r['type'] == 'table' else "📄"
        src = r['source']
        print(f"  {marker} #{r['rank']} | {icon} {r['type']:6s} | {src:12s} | {r['score']:6.2f} | {r['doc_id']}")

        if r['doc_id'] == expected_doc and r['rank'] <= 3:
            print(f"       {r['content'][:120]}...")

    retrieved_docs = get_doc_from_chunks(results)
    if retrieved_docs[0] == expected_doc:
        print(f"\n  ✅ 成功！")
    else:
        print(f"\n  ❌ 失敗")
    print()

print("="*70)
print("📊 三種方法對比")
print("="*70)
print("\n方法 1 - 純文字 chunks:")
print("  Recall@1: 85.37%  |  Recall@5: 92.68%  |  MRR: 0.8882")

print("\n方法 2 - 混合 chunks (失敗):")
print("  Recall@1: 46.34%  |  Recall@5: 60.98%  |  MRR: 0.5305")

print(f"\n方法 3 - 智能混合檢索 (當前):")
print(f"  Recall@1: {metrics['Recall@1']:.2%}  |  Recall@5: {metrics['Recall@5']:.2%}  |  MRR: {metrics['MRR']:.4f}")

print("\n✨ 推薦使用：智能混合檢索")
print("="*70)

📂 載入 chunks...

✓ 純文字 chunks: 1778
✓ 混合 chunks: 2608

🔨 建立雙重 BM25 索引...

✓ 純文字 BM25 索引完成
✓ 表格 BM25 索引完成 (134 個表格)

📊 評估混合檢索策略...

  ✓ 完成 41 問題                    

📈 混合檢索效果（智能路由）
評估問題數: 41
表格查詢數: 27 (65.9%)

  Recall@ 1  : 85.37%  ██████████████████████████████████████████
  Recall@ 3  : 90.24%  █████████████████████████████████████████████
  Recall@ 5  : 90.24%  █████████████████████████████████████████████
  Recall@10  : 90.24%  █████████████████████████████████████████████

  MRR        : 0.8780

🔍 測試關鍵案例...

──────────────────────────────────────────────────────────────────────
📋 q054: What is the model size in gigabytes (GB) for the LLaMA-33B model?...
✓ 期望: chen2024
🔍 查詢類型: 表格查詢 📊

檢索結果:
  ✅ #1 | 📄 text   | text_index   |  27.36 | chen2024
       For larger payload sizes, the primary factor influencing networking time is the utilization of network bandwidth. In thi...
     #2 | 📄 text   | text_index   |  27.19 | erben2023
     #3 | 📄 text   | text_index   |  26.59 | rubei2025
  ✅

### **3.1 Run1 — Sparse Retrieval (BM25)**

In [ ]:
def evaluate_system(
    qa_df: pd.DataFrame,
    search_function,
    system_name: str,
    **search_kwargs
) -> Dict:
    print(f"評估 {system_name}...")

    recalls = {k: [] for k in [1, 3, 5, 10]}
    ndcgs = {k: [] for k in [1, 3, 5, 10]}
    mrr_scores = []

    for idx, row in qa_df.iterrows():
        question = row['question']
        ref_id_raw = row['ref_id']

        if pd.isna(ref_id_raw) or str(ref_id_raw).strip() == '':
            continue
        try:
            if isinstance(ref_id_raw, str):
                if ref_id_raw.startswith('['):
                    true_docs = ast.literal_eval(ref_id_raw)
                    if not isinstance(true_docs, list):
                        true_docs = [true_docs]
                else:
                    true_docs = [ref_id_raw.strip()]
            else:
                true_docs = [str(ref_id_raw).strip()]
        except:
            continue

        # 跳過trick questions
        is_trick = any(doc in ['is_blank', ''] for doc in true_docs)
        if is_trick:
            continue

        results = search_function(question, **search_kwargs)

        retrieved_docs = []
        seen = set()
        for r in results:
            doc_id = r['doc_id']
            if doc_id not in seen:
                seen.add(doc_id)
                retrieved_docs.append(doc_id)

        # 計算Recall@k
        for k in [1, 3, 5, 10]:
            top_k = retrieved_docs[:k]
            hit = any(doc in top_k for doc in true_docs)
            recalls[k].append(1 if hit else 0)

        # 計算MRR
        reciprocal_rank = 0.0
        for rank, doc in enumerate(retrieved_docs, 1):
            if doc in true_docs:
                reciprocal_rank = 1.0 / rank
                break
        mrr_scores.append(reciprocal_rank)

        # 計算nDCG@k
        for k in [1, 3, 5, 10]:
            ndcg = calculate_ndcg(retrieved_docs, true_docs, k)
            ndcgs[k].append(ndcg)

    metrics = {}
    for k in [1, 3, 5, 10]:
        metrics[f'Recall@{k}'] = np.mean(recalls[k]) if recalls[k] else 0.0
        metrics[f'nDCG@{k}'] = np.mean(ndcgs[k]) if ndcgs[k] else 0.0
    metrics['MRR'] = np.mean(mrr_scores) if mrr_scores else 0.0
    metrics['num_queries'] = len(recalls[1])

    print(f"  ✓ 完成 ({metrics['num_queries']} queries)")

    return metrics

In [ ]:
import re
import numpy as np
import pandas as pd
from typing import List, Dict
from rank_bm25 import BM25Okapi
import ast
# 注意：此模組依賴您在其他 Cell 中載入的 train_qa 和 chunks 變數（chunks = hybrid_chunks）

# ============================================
# 1. BM25 核心函數定義
# ============================================

def enhanced_tokenize(text: str) -> List[str]:
    """增強分詞器：保留數字、年份、技術術語、bigrams"""
    # 這是您決定保留的 Tokenizer 邏輯
    text = text.lower()
    tokens = []

    numbers = re.findall(r'\d+\.?\d*%?|\d+[kmbt]?', text)
    tokens.extend(numbers)

    years = re.findall(r'\b20\d{2}\b', text)
    tokens.extend(years)

    tech_terms = re.findall(
        r'\b(?:gpu|tpu|cpu|kwh|mwh|twh|co2|tco2e|gflops|teraflops|'
        r'bert|gpt|llama|transformer|bm25|household|consumption)\b',
        text
    )
    tokens.extend(tech_terms)

    words = re.findall(r'[a-z]+', text)
    tokens.extend(words)

    bigrams = [f"{words[i]}_{words[i+1]}" for i in range(len(words)-1)]
    tokens.extend(bigrams[:5])

    return tokens

def build_bm25_enhanced(chunks: List[Dict]):
    """建立 BM25 索引物件 - ⭐ 已切換到 Hybrid Chunks"""
    print("建立 BM25 Enhanced 索引...")

    # ⭐ 關鍵修改：將 corpus 來源切換到 'chunks' (hybrid data)
    corpus = [chunk['content'] for chunk in chunks]
    tokenized_corpus = [enhanced_tokenize(text) for text in corpus]

    bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)
    print(f"✓ 完成 ({len(corpus)} chunks)")
    return bm25

def search_bm25_enhanced(query: str, bm25: BM25Okapi, chunks: List[Dict], top_k: int = 10) -> List[Dict]:
    """
    執行 BM25 檢索 (被 evaluate_system 呼叫) - ⭐ 已切換到 Hybrid Chunks

    注意：此處依賴的 chunks 參數已在 evaluate_system 中傳入
    """
    tokenized_query = enhanced_tokenize(query)
    scores = bm25.get_scores(tokenized_query)

    top_indices = np.argsort(scores)[-top_k:][::-1]

    results = []
    for rank, idx in enumerate(top_indices, 1):
        # ⭐ 關鍵修改：從傳入的 chunks 參數中獲取內容
        results.append({
            'rank': rank,
            'score': float(scores[idx]),
            'doc_id': chunks[idx]['doc_id'],
            'content': chunks[idx]['content'],
            'source': 'bm25_enhanced'
        })
    return results
def calculate_ndcg(retrieved_docs: List[str], true_docs: List[str], k: int) -> float:
    """計算nDCG@k"""
    # DCG (Discounted Cumulative Gain)
    dcg = 0.0
    for i, doc in enumerate(retrieved_docs[:k], 1):
        relevance = 1 if doc in true_docs else 0
        dcg += relevance / np.log2(i + 1)

    # IDCG (Ideal DCG)
    ideal_docs = true_docs + ['dummy'] * k
    idcg = 0.0
    for i in range(1, min(len(true_docs), k) + 1):
        idcg += 1.0 / np.log2(i + 1)

    # nDCG
    if idcg == 0:
        return 0.0
    return dcg / idcg

# ============================================
# 2. 建立索引 (讓 bm25_enhanced 變數生效)
# ============================================
print("="*70)
print("Run1: BM25 Enhanced 核心模組")
print("="*70)
bm25_enhanced = build_bm25_enhanced(chunks) # ⭐ 傳入 chunks
print()

# ============================================
# 3. 評估與結果展示 (修正後的執行順序)
# ============================================

# ⭐ 關鍵：evaluate_system 必須傳入 chunks 變數
metrics_run1 = evaluate_system(
    train_qa,
    search_bm25_enhanced,
    "Run1: BM25 Enhanced (Hybrid Data)",
    bm25=bm25_enhanced,
    chunks=chunks, # ⭐ 傳入 chunks 列表
    top_k=10
)

print()
print("="*70)
print("📊 評估結果 (Run 1: BM25 Enhanced)")
print("="*70)
print()

# ... (後續的 DataFrame 顯示邏輯) ...

Run1: BM25 Enhanced 核心模組
建立 BM25 Enhanced 索引...
✓ 完成 (63 chunks)

評估 Run1: BM25 Enhanced (Hybrid Data)...
  ✓ 完成 (39 queries)

📊 評估結果 (Run 1: BM25 Enhanced)



In [ ]:
# -----------------------------------------------------------
# I. 定義雙重索引和搜索函數 (假設 text_chunks & chunks 均已載入)
# -----------------------------------------------------------
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from typing import List, Dict

# 注意：這些函數依賴於您前面定義的 enhanced_tokenize
# 假設 enhanced_tokenize, calculate_ndcg, evaluate_system, train_qa 已經可用

def build_bm25_index_for_comparison(data_list: List[Dict]):
    """根據傳入的列表建立 BM25 索引"""
    corpus = [chunk['content'] for chunk in data_list]
    tokenized_corpus = [enhanced_tokenize(text) for text in corpus]
    return BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)

def search_bm25_generic(query: str, bm25_index: BM25Okapi, data_list: List[Dict], top_k: int = 10) -> List[Dict]:
    """通用 BM25 檢索函數 (傳入 index 和 data source)"""
    tokenized_query = enhanced_tokenize(query)
    scores = bm25_index.get_scores(tokenized_query)

    top_indices = np.argsort(scores)[-top_k:][::-1]

    results = []
    for rank, idx in enumerate(top_indices, 1):
        chunk = data_list[idx]
        results.append({
            'rank': rank,
            'score': float(scores[idx]),
            'doc_id': chunk['doc_id'],
            'content': chunk['content'],
            'source': 'bm25_comp'
        })
    return results

# -----------------------------------------------------------
# II. 建立兩個獨立的索引
# -----------------------------------------------------------
print("="*70)
print("🔨 建立雙重索引...")
print("="*70)
chunks = hybrid_chunks
# 1. Hybrid Data Index
bm25_hybrid = build_bm25_index_for_comparison(chunks) # 假設 chunks = hybrid_chunks
print(f"✓ Hybrid Index 完成 ({len(chunks)} chunks)")

# 2. Text-Only Index
bm25_text = build_bm25_index_for_comparison(text_chunks) # 假設 text_chunks = 舊純文本
print(f"✓ Text-Only Index 完成 ({len(text_chunks)} chunks)")


# -----------------------------------------------------------
# III. 執行雙重評估
# -----------------------------------------------------------
print("\n🔄 執行雙重評估 (Run Hybrid vs Run Text-Only)")

# 評估 Run A (Hybrid Data - 結構化新數據源)
metrics_hybrid = evaluate_system(
    train_qa,
    search_bm25_generic,
    "Run A: Hybrid Data (NEW)",
    bm25_index=bm25_hybrid,
    data_list=chunks,
    top_k=10
)

print()

# 評估 Run B (Text-Only - 扁平化舊數據源)
metrics_text = evaluate_system(
    train_qa,
    search_bm25_generic,
    "Run B: Text-Only (OLD)",
    bm25_index=bm25_text,
    data_list=text_chunks,
    top_k=10
)


# -----------------------------------------------------------
# IV. 顯示最終對比表格
# -----------------------------------------------------------
print("\n" + "="*70)
print("📊 BM25 資料源對比結果")
print("="*70)

comparison_data = []
for metric in ['Recall@1', 'Recall@3', 'MRR', 'nDCG@5']:
    r_hybrid = metrics_hybrid[metric]
    r_text = metrics_text[metric]

    best_val = max(r_hybrid, r_text)

    comparison_data.append({
        'Metric': metric,
        'Hybrid Data (A)': f"{r_hybrid:.4f}" + ("*" if r_hybrid == best_val else ""),
        'Text Only (B)': f"{r_text:.4f}" + ("*" if r_text == best_val else ""),
        'Delta (A-B)': f"{r_hybrid - r_text:+.4f}"
    })

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

print("\n" + "─"*70)
if metrics_hybrid['Recall@1'] >= metrics_text['Recall@1']:
    print("✅ 結論：Hybrid Data 的 Recall 佔優或持平。維持使用 Hybrid Data。")
else:
    print("⚠️ 結論：Text-Only Data  Recall 仍較高。需要分析 Hybrid Chunks 移除的資料。")
print("="*70)

🔨 建立雙重索引...
✓ Hybrid Index 完成 (2608 chunks)
✓ Text-Only Index 完成 (1778 chunks)

🔄 執行雙重評估 (Run Hybrid vs Run Text-Only)
評估 Run A: Hybrid Data (NEW)...
  ✓ 完成 (39 queries)

評估 Run B: Text-Only (OLD)...
  ✓ 完成 (39 queries)

📊 BM25 資料源對比結果
  Metric Hybrid Data (A) Text Only (B) Delta (A-B)
Recall@1          0.7949       0.8718*     -0.0769
Recall@3          0.8974       0.9231*     -0.0256
     MRR          0.8526       0.8974*     -0.0449
  nDCG@5          0.8564       0.8859*     -0.0295

──────────────────────────────────────────────────────────────────────
⚠️ 結論：Text-Only Data  Recall 仍較高。需要分析 Hybrid Chunks 移除的資料。


In [ ]:
import pandas as pd
test_q = pd.read_csv(INPUT_DIR / "test_Q.csv", encoding="latin1")
print("欄位名稱:", test_q.columns.tolist())


欄位名稱: ['ï»¿id', 'question', 'answer', 'answer_value', 'answer_unit', 'ref_id', 'ref_url', 'supporting_materials', 'explanation']


# AG

In [ ]:
import os
import re
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from tqdm.auto import tqdm
from rank_bm25 import BM25Okapi
from collections import defaultdict
from datetime import datetime, timedelta
from google import genai
from google.genai import types

# ============================================================================
# 0. API 設定
# ============================================================================

GEMINI_API_KEY = "# ⚠️ 請替換成你的 API Key"  # ⚠️ 請替換成你的 API Key
if not GEMINI_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY 環境變數沒設到")

client = genai.Client(api_key=GEMINI_API_KEY)

MODEL_CONFIGS = {
    "gemini-2.5-flash": {
        "input_price": 0.075,
        "output_price": 0.30,
        "rpm_limit": 1000,
        "description": "Gemini 2.5 Flash - 高速 (RPM 1000)"
    },
    "gemini-2.5-pro": {
        "input_price": 1.25,
        "output_price": 5.0,
        "rpm_limit": 150,
        "description": "Gemini 2.5 Pro - 高品質 (RPM 150)"
    },
    "gemini-3-pro-preview": {
        "input_price": 2.0,
        "output_price": 12.0,
        "rpm_limit": 50,
        "description": "Gemini 3.0 Pro Preview - 次世代 (RPM 50)"
    },
    "gemini-2.0-flash": {
        "input_price": 0.10,
        "output_price": 0.40,
        "rpm_limit": 2000,
        "description": "Gemini 2.0 Flash (RPM 2000)"
    }
}


# ============================================================================
# 1. 問題類型檢測器
# ============================================================================

def detect_question_type(question: str) -> Tuple[str, bool]:
    """
    檢測問題類型，返回 (問題類型, 是否需要激進策略)

    Returns:
        (question_type, needs_aggressive_mode)
    """
    q_lower = question.lower()

    # 類型 1: 表格數值提取 (最容易失敗)
    table_extraction_patterns = [
        r'execution time',
        r'total time',
        r'throughput.*queries',
        r'queries.*per.*sec',
        r'batch size',
        r'maximum batch',
        r'gpu hours',
        r'h100.*hours',
        r'a100.*hours',
        r'total.*parameters',
        r'number of parameters',
        r'number of experts',
        r'how many experts',
        r'energy consumption.*wh',
        r'power usage.*kwh',
        r'cost.*\$|cost.*usd',
        r'price per hour',
        r'energy.*mwh|energy.*kwh',
        r'what is the .* for .* model',
    ]
    for pattern in table_extraction_patterns:
        if re.search(pattern, q_lower):
            return ("table_extraction", True)

    # 類型 2: 特定模型/論文的數據
    specific_model_patterns = [
        (r'jetmoe', "model_specific"),
        (r'blackmamba', "model_specific"),
        (r'mixtral', "model_specific"),
        (r'flm-101b', "model_specific"),
        (r'bloom\b', "model_specific"),
        (r'olmo', "model_specific"),
        (r'llama.?\d', "model_specific"),
    ]
    for pattern, qtype in specific_model_patterns:
        if re.search(pattern, q_lower):
            return (qtype, True)

    # 類型 3: 分數/排名問題
    score_patterns = [
        r'what score',
        r'average score',
        r'final.* score',
        r'benchmark.*score',
        r'accuracy',
        r'performance score',
        r'leaderboard',
        r'gsm8k',
        r'mt-bench',
        r'bleu',
    ]
    for pattern in score_patterns:
        if re.search(pattern, q_lower):
            return ("score_extraction", True)

    # 類型 4: 術語/名稱問題
    term_patterns = [
        r'what is the name of',
        r'what is the term',
        r'which (?:software|framework|tool|package|metric)',
        r'what (?:software|framework|tool|package|metric)',
        r'what.*called',
    ]
    for pattern in term_patterns:
        if re.search(pattern, q_lower):
            return ("term_extraction", True)

    # 類型 5: 數量/計數問題 (通常有明確答案)
    count_patterns = [
        r'how many',
        r'how much',
        r'what percentage',
        r'what fraction',
        r'what proportion',
        r'by what factor',
        r'what was the .* increase',
        r'what was the .* decrease',
    ]
    for pattern in count_patterns:
        if re.search(pattern, q_lower):
            return ("numeric_extraction", True)

    return ("general", False)


def detect_multi_doc_question(question: str) -> Tuple[bool, str]:
    """保守地偵測是否需要多篇論文"""
    question_lower = question.lower()

    cross_calc_patterns = [
        r'equivalent to .+ (household|american|person|life)',
        r'how many .+ equivalent to',
        r'compare.*to',
    ]
    for pattern in cross_calc_patterns:
        if re.search(pattern, question_lower):
            return True, "cross_doc_calculation"

    if 'and how does this compare' in question_lower:
        return True, "explicit_comparison"

    return False, "single_doc"


# ============================================================================
# 2. 增強版表格解析 Prompt
# ============================================================================

def get_enhanced_table_prompt() -> str:
    """返回增強版的表格解析指引"""
    return """
📊 ENHANCED TABLE PARSING RULES (CRITICAL):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. TABLE STRUCTURE:
   - Tables use "|" to separate columns
   - First row = HEADER, subsequent rows = DATA
   - Example:
     | Model | Batch Size | Time (s) | Cost ($) |
     |---------|---------------|------------|-------------|
     | Dense |     30  |   45.2 |   12.5  |
     | Sparse |     84  |   32.1 |    8.7 |

2. HOW TO READ:
   - STEP 1: Find ROW matching question criteria (model, config)
   - STEP 2: Find COLUMN containing the answer type
   - STEP 3: Extract VALUE at intersection
   - STEP 4: Include UNIT from column header

3. CRITICAL MATCHING:
   - "dense" vs "sparse" → DIFFERENT rows!
   - "BlackMamba" vs "Mixtral" → DIFFERENT models!
   - "A40" vs "A100" vs "H100" → DIFFERENT GPUs!
   - "batch size = 1" vs "batch size = 30" → DIFFERENT rows!

4. IF YOU SEE A TABLE:
   - Check ALL rows before saying "Unable to answer"
   - The answer IS there if the model/config is mentioned
   - Extract EXACT number with full precision
"""


# ============================================================================
# 3. BM25 檢索模組
# ============================================================================

def enhanced_tokenize(text: str) -> List[str]:
    """增強分詞器：保留數字、年份、技術術語"""
    text = text.lower()
    tokens = []

    # 數字 (含小數、百分比)
    tokens.extend(re.findall(r'\d+\.?\d*%?', text))

    # 年份
    tokens.extend(re.findall(r'\b20\d{2}\b', text))

    # 技術術語
    tokens.extend(re.findall(
        r'\b(?:gpu|tpu|cpu|kwh|mwh|twh|co2|tco2e|gflops|teraflops|'
        r'bert|gpt|llama|transformer|household|consumption|'
        r'jetmoe|blackmamba|mixtral|bloom|inference|training)\b',
        text
    ))

    # 一般單詞
    words = re.findall(r'[a-z]+', text)
    tokens.extend(words)

    # Bigrams (前5個)
    tokens.extend([f"{words[i]}_{words[i+1]}" for i in range(min(5, len(words)-1))])

    return tokens


def build_bm25_index(chunks: List[Dict]) -> BM25Okapi:
    """建立 BM25 索引"""
    print("\n🔨 建立 BM25 索引...")
    corpus = [chunk['content'] for chunk in chunks]
    tokenized_corpus = [enhanced_tokenize(text) for text in corpus]
    bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)
    print(f"✓ BM25 索引完成 ({len(corpus)} chunks)")
    return bm25


def search_bm25(query: str, bm25: BM25Okapi, chunks: List[Dict], top_k: int = 10) -> List[Dict]:
    """BM25 檢索"""
    tokenized_query = enhanced_tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[-top_k:][::-1]

    results = []
    for rank, idx in enumerate(top_indices, 1):
        chunk = chunks[idx]
        results.append({
            'rank': rank,
            'score': float(scores[idx]),
            'doc_id': chunk['doc_id'],
            'content': chunk['content'],
            'type': chunk.get('type', 'text')
        })

    return results


# ============================================================================
# 4. 增強版 Answer Generator (Gemini API)
# ============================================================================

class EnhancedTwoStageGPTGenerator:
    """
    增強版兩階段生成器 - Gemini API

    改進:
    1. 問題類型檢測 → 自動選擇激進/保守策略
    2. 增強表格解析 prompt
    3. 對特定問題類型降低 "Unable to answer" 門檻
    """


    def __init__(self, model: str = "gemini-2.5-pro", client=None, rpm_limit: int = None):
        self.model = model
        self.client = client
        self.call_count = 0
        self.total_cost = 0.0
        self.stage1_success = 0
        self.stage2_retry = 0
        self.multi_doc_count = 0
        self.aggressive_count = 0
        self.no_text_count = 0

        # 此函數是為了讓Gemini能快速切換模型才建立
        if model in MODEL_CONFIGS:
            self.model_config = MODEL_CONFIGS[model]
            self.rpm_limit = rpm_limit or self.model_config['rpm_limit']
            print(f"📌 使用模型: {model}")
            print(f"   {self.model_config['description']}")
            print(f"   定價: ${self.model_config['input_price']}/${self.model_config['output_price']} per 1M tokens")
            print(f"   Rate Limit: {self.rpm_limit}/min")
        else:
            print(f"⚠️ 未知模型 {model}，使用默認配置")
            self.model_config = MODEL_CONFIGS["gemini-2.5-flash"]
            self.rpm_limit = rpm_limit or 150

        self.call_timestamps = []

    #為了應對Gemini API 每分鐘請求上限
    def _wait_for_rate_limit(self):
        """確保不超過rate limit"""
        now = datetime.now()

        self.call_timestamps = [
            ts for ts in self.call_timestamps
            if now - ts < timedelta(minutes=1)
        ]

        if len(self.call_timestamps) >= self.rpm_limit:
            oldest_call = min(self.call_timestamps)
            wait_time = 60 - (now - oldest_call).total_seconds()
            if wait_time > 0:
                print(f"  ⏳ Rate limit, waiting {wait_time:.1f}s...")
                time.sleep(wait_time + 1)

        self.call_timestamps.append(datetime.now())

    def generate_answer(
        self,
        question: str,
        retrieval_results: List[Dict],
        expected_unit: str = "is_blank"
    ) -> Dict:
        """兩階段生成主函數"""

        # 問題類型檢測
        question_type, needs_aggressive = detect_question_type(question)
        if needs_aggressive:
            self.aggressive_count += 1

        # 多文檔檢測
        is_multi_doc, reason = detect_multi_doc_question(question)
        if is_multi_doc:
            self.multi_doc_count += 1

        # ============================================
        # Stage 1: Top-3 + 根據問題類型選擇策略
        # ============================================
        top3_results = retrieval_results[:3]
        context_top3 = self._build_context(top3_results)

        stage1_response = self._call_gpt(
            question=question,
            context=context_top3,
            retrieval_results=top3_results,
            expected_unit=expected_unit,
            mode="aggressive" if needs_aggressive else "strict",
            question_type=question_type,
            is_multi_doc=is_multi_doc
        )

        is_unable = "Unable to answer" in stage1_response.get('answer', '')

        if not is_unable:
            self.stage1_success += 1
            return stage1_response

        # ============================================
        # Stage 2: 擴大範圍 + 更激進的策略
        # ============================================
        self.stage2_retry += 1

        # 對特定問題類型使用更多 context
        top_k = 10 if needs_aggressive else 7
        top_results = retrieval_results[:top_k]
        context_expanded = self._build_context(top_results)

        stage2_response = self._call_gpt(
            question=question,
            context=context_expanded,
            retrieval_results=top_results,
            expected_unit=expected_unit,
            mode="aggressive",  # Stage 2 永遠使用激進模式
            question_type=question_type,
            is_multi_doc=is_multi_doc
        )

        return stage2_response

    def _build_context(self, results: List[Dict]) -> str:
        """構建上下文 - 增強表格標記"""
        context_parts = []

        for i, r in enumerate(results, 1):
            content = r['content']
            chunk_type = r.get('type', 'text')
            doc_id = r['doc_id']

            if chunk_type == 'table':
                prefix = (
                    f"[DOCUMENT #{i}: {doc_id}]\n"
                    f"⚠️ DATA TABLE - READ CAREFULLY:\n"
                    f"{'='*60}\n"
                )
            else:
                prefix = f"[DOCUMENT #{i}: {doc_id}]\n"

            context_parts.append(prefix + content)

        return "\n\n".join(context_parts)

    # 因為Gemini 每版本的JSON輸出模式不同，因此設置此函數抓取正確回覆來源
    def _extract_text_from_response(self, response) -> Optional[str]:
        """從 Gemini 回應中提取文字，兼容 2.5 / 3.0"""
        try:
            # 路徑 1：最簡單，先試 response.text
            try:
                text = response.text
            except Exception:
                text = None

            if isinstance(text, str) and text.strip():
                return text.strip()

            # 路徑 2：Gemini 3 推薦的 response.parts
            if hasattr(response, "parts") and response.parts:
                text_parts = []
                for part in response.parts:
                    part_text = getattr(part, "text", None)
                    if isinstance(part_text, str) and part_text.strip():
                        text_parts.append(part_text.strip())
                if text_parts:
                    return "\n".join(text_parts)

            # 路徑 3：舊版格式 candidates[0].content.parts
            if hasattr(response, "candidates") and response.candidates:
                for cand in response.candidates:
                    content = getattr(cand, "content", None)
                    if content is None:
                        continue

                    parts = getattr(content, "parts", None)
                    if not parts:
                        continue

                    text_parts = []
                    for part in parts:
                        part_text = getattr(part, "text", None)
                        if isinstance(part_text, str) and part_text.strip():
                            text_parts.append(part_text.strip())

                    if text_parts:
                        return "\n".join(text_parts)

            return None

        except Exception as e:
            print(f"  ⚠️ 提取文本異常: {e}")
            return None

    def _call_gpt(
        self,
        question: str,
        context: str,
        retrieval_results: List[Dict],
        expected_unit: str,
        mode: str,
        question_type: str,
        is_multi_doc: bool
    ) -> Dict:
        """調用 Gemini API"""

        if mode == "aggressive":
            system_prompt = self._get_aggressive_system_prompt(question_type, is_multi_doc)
        else:
            system_prompt = self._get_strict_system_prompt(is_multi_doc)

        user_prompt = self._build_user_prompt(
            question, context, expected_unit, retrieval_results, question_type, is_multi_doc
        )

        max_retries = 3
        retry_count = 0

        while retry_count < max_retries:
            try:
                self._wait_for_rate_limit()

                full_prompt = f"""{system_prompt}

---

{user_prompt}"""

                # Safety Settings，此段落為新增，讓 Gemini 不要過濾問題
                safety_settings = [
                    types.SafetySetting(
                        category="HARM_CATEGORY_HATE_SPEECH",
                        threshold="BLOCK_NONE"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold="BLOCK_NONE"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                        threshold="BLOCK_NONE"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_HARASSMENT",
                        threshold="BLOCK_NONE"
                    )
                ]

                # Generation Config
                gen_config = types.GenerateContentConfig(
                    temperature=0.1,
                    max_output_tokens=8192,
                    response_mime_type="application/json",
                    safety_settings=safety_settings
                )

                response = self.client.models.generate_content(
                    model=self.model,
                    contents=full_prompt,
                    config=gen_config,
                )

                self.call_count += 1

                content = self._extract_text_from_response(response)

                # 處理無文本回應
                if content is None or content.strip() == '':
                    self.no_text_count += 1

                    # 抓取被擋掉的原因
                    reason = "UNKNOWN"
                    try:
                        if response.candidates:
                            reason = response.candidates[0].finish_reason
                    except:
                        pass

                    print(f"  ⚠️ 無文本 (原因: {reason}) (累計 {self.no_text_count})")

                    # 如果是版權問題，直接跳過
                    if str(reason) == "RECITATION":
                        print("  🛑 偵測到版權引用限制 (Recitation)，跳過重試")
                        return self._default_answer(expected_unit)

                    if retry_count < max_retries - 1:
                        print(f"  🔄 重試...")
                        retry_count += 1
                        time.sleep(3)
                        continue
                    else:
                        return self._default_answer(expected_unit)

                result = self._parse_response(content, expected_unit)

                # 計算成本
                try:
                    input_tokens = response.usage_metadata.prompt_token_count
                    output_tokens = response.usage_metadata.candidates_token_count
                    input_price = self.model_config['input_price']
                    output_price = self.model_config['output_price']
                    cost = (input_tokens * input_price + output_tokens * output_price) / 1_000_000
                    self.total_cost += cost
                except:
                    self.total_cost += 0.001

                return result

            except Exception as e:
                error_str = str(e)
                if "429" in error_str or "RESOURCE_EXHAUSTED" in error_str:
                    retry_count += 1
                    if retry_count < max_retries:
                        wait_time = min(10 * (2 ** retry_count), 60)
                        print(f"  ⏳ Rate limit (retry {retry_count}/{max_retries}), waiting {wait_time}s...")
                        time.sleep(wait_time)
                        continue

                print(f"  ❌ API 錯誤: {e}")
                if retry_count < max_retries - 1:
                    retry_count += 1
                    continue
                return self._default_answer(expected_unit)

        return self._default_answer(expected_unit)

#此段與原版相同
    def _get_aggressive_system_prompt(self, question_type: str, is_multi_doc: bool) -> str:
        """激進模式 System Prompt"""

        table_section = get_enhanced_table_prompt()

        multi_doc_section = """
⭐ MULTI-DOCUMENT CITATION:
- Combine data from multiple documents if needed
- List all sources: ["doc1", "doc2"]
""" if is_multi_doc else ""

        # 問題類型特定提示
        type_hints = {
            "table_extraction": """
🎯 TABLE EXTRACTION MODE:
- The answer IS in a table - search ALL rows carefully!
- Match: model name + GPU type + batch size + dense/sparse
- DO NOT say "Unable to answer" if you see relevant tables!
""",
            "model_specific": """
🎯 MODEL-SPECIFIC MODE:
- Look for data specific to this MODEL
- Common data: parameter counts, expert counts, scores, GPU hours, tokens
- If model name appears in context, the answer should be there!
""",
            "score_extraction": """
🎯 SCORE EXTRACTION MODE:
- Look for benchmark tables or score mentions
- Common: GSM8k, MT-Bench, OpenLLM Leaderboard, accuracy, BLEU
- Extract the EXACT number with full precision
""",
            "term_extraction": """
🎯 TERM/NAME EXTRACTION MODE:
- Look for definitions, tool names, framework names, metrics
- The answer is a proper noun or technical term
- Search for quoted or emphasized text
""",
            "numeric_extraction": """
🎯 NUMERIC EXTRACTION MODE:
- Look for specific numbers, percentages, counts
- The answer exists in the documents
- Extract the exact value mentioned
""",
        }

        type_hint = type_hints.get(question_type, "")

        return f"""⚠️ JSON OUTPUT RULES:
- Output a single valid JSON object
- ALL fields: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
- Use "is_blank" for empty values (never null)
- source_doc_ids = LIST: ["doc_id"]

{type_hint}
{table_section}
{multi_doc_section}

⚠️ AGGRESSIVE MODE - REDUCED "UNABLE TO ANSWER" THRESHOLD:
- Only say "Unable to answer" if CERTAIN the info doesn't exist
- If you see ANY relevant data, extract it!
- Partial matches acceptable - use best available info
- For numerical questions, extract the closest matching value
- DO NOT refuse to answer if you see the model/topic mentioned!

🔢 UNIT HANDLING:
- MWh → kWh: ×1000
- kWh → Wh: ×1000
- million → raw: ×1,000,000
- billion → raw: ×1,000,000,000
- Always check if conversion is needed!

🧠 EXPLANATION:
- For table data: "Found in row X, column Y of table"
- Show calculations if needed
- Cite the specific text/number you found

You are a data extraction expert. PREFER ANSWERING over refusing."""

    def _get_strict_system_prompt(self, is_multi_doc: bool) -> str:
        """嚴格模式 System Prompt"""

        table_section = get_enhanced_table_prompt()

        multi_doc_section = """
⭐ MULTI-DOCUMENT CITATION:
- List sources as: ["doc1", "doc2"]
""" if is_multi_doc else ""

        return f"""⚠️ JSON OUTPUT RULES:
- Output a single valid JSON object
- ALL fields: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
- Use "is_blank" for empty values
- source_doc_ids = LIST

⚠️ UNABLE TO ANSWER:
- If cannot answer: answer = "Unable to answer with confidence based on the provided documents."
- answer_value = "is_blank", source_doc_ids = []
- DO NOT guess or fabricate
{multi_doc_section}
{table_section}

🔢 UNIT CONVERSION:
- MWh → kWh: ×1000
- million → raw: ×1,000,000

✅ STRICT MODE: Answer only if evidence is explicit and clear.

You are a scientific evidence extraction expert."""

    def _build_user_prompt(
        self,
        question: str,
        context: str,
        expected_unit: str,
        retrieval_results: List[Dict],
        question_type: str,
        is_multi_doc: bool
    ) -> str:
        """構建 User Prompt"""

        available_doc_ids = list(set([r['doc_id'] for r in retrieval_results]))
        doc_id_list = ", ".join(available_doc_ids)

        # 問題類型提示
        type_hints = {
            "table_extraction": "⚠️ HINT: Answer is in a TABLE. Search all rows carefully!",
            "model_specific": "⚠️ HINT: Look for this specific MODEL's data in the context.",
            "score_extraction": "⚠️ HINT: Look for benchmark scores or performance metrics.",
            "term_extraction": "⚠️ HINT: Look for technical terms, tool names, or definitions.",
            "numeric_extraction": "⚠️ HINT: Look for specific numbers or percentages.",
        }
        hint = type_hints.get(question_type, "")

        multi_hint = "\n⚠️ May need to combine data from multiple documents." if is_multi_doc else ""

        return f"""QUESTION:
{question}

{hint}{multi_hint}

EXPECTED UNIT: {expected_unit}

AVAILABLE DOC IDs: {doc_id_list}

CONTEXT:
{context[:12000]}

OUTPUT FORMAT (STRICT JSON):
{{
  "answer": "natural language answer with unit",
  "answer_value": "number or keyword (NO unit, NO symbols like <, >, ~)",
  "answer_unit": "{expected_unit}",
  "supporting_text": "verbatim quote from context (≤30 words)",
  "explanation": "step-by-step reasoning (mention row/col for tables)",
  "source_doc_ids": ["doc_id"]
}}

SPECIAL CASES:
- TRUE/FALSE: answer_value = "1" (TRUE) or "0" (FALSE)
- Range: answer_value = "[low,high]" (no spaces inside brackets)
- Unable to answer: ONLY if you searched ALL context and found NOTHING relevant

GENERATE JSON NOW:"""

    def _parse_response(self, content: str, expected_unit: str) -> Dict:
        """解析 Gemini 響應"""
        try:
            content = content.replace('```json', '').replace('```', '').strip()
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if not json_match:
                return self._default_answer(expected_unit)

            result = json.loads(json_match.group())

            # 確保必要欄位
            required_fields = ['answer', 'answer_value', 'answer_unit',
                             'supporting_text', 'explanation', 'source_doc_ids']
            for field in required_fields:
                if field not in result or result[field] is None:
                    if field == 'source_doc_ids':
                        result[field] = []
                    else:
                        result[field] = 'is_blank'

            # 處理 source_doc_ids
            doc_ids = result.get('source_doc_ids', [])
            if isinstance(doc_ids, str):
                if doc_ids == 'is_blank' or not doc_ids:
                    result['source_doc_ids'] = []
                else:
                    result['source_doc_ids'] = [doc_ids]
            elif not isinstance(doc_ids, list):
                result['source_doc_ids'] = []

            result['source_doc_ids'] = list(set(result['source_doc_ids']))

            # ✅ 修正：不區分大小寫檢查 "unable to answer"
            answer_text = result.get('answer', '').lower()
            if 'unable to answer' in answer_text or 'cannot answer' in answer_text or 'not able to answer' in answer_text:
                return {
                    'answer': 'Unable to answer with confidence based on the provided documents.',
                    'answer_value': 'is_blank',
                    'answer_unit': expected_unit,
                    'supporting_text': 'is_blank',
                    'explanation': 'is_blank',
                    'source_doc_ids': []
                }

            # TRUE/FALSE 處理
            answer_str = str(result.get('answer', '')).strip()
            answer_val = str(result.get('answer_value', '')).strip().upper()

            # 檢查多種 TRUE/FALSE 格式
            if answer_str.upper() in ['TRUE', 'FALSE'] or answer_val in ['TRUE', 'FALSE']:
                is_true = answer_str.upper() == 'TRUE' or answer_val == 'TRUE'
                result['answer'] = 'TRUE' if is_true else 'FALSE'
                result['answer_value'] = '1' if is_true else '0'
                result['answer_unit'] = 'is_blank'
            else:
                result['answer_unit'] = expected_unit

            # 清理空值
            for key in result:
                if result[key] is None or result[key] == 'null':
                    if key == 'source_doc_ids':
                        result[key] = []
                    else:
                        result[key] = 'is_blank'

            return result

        except Exception as e:
            print(f"  解析錯誤: {e}")
            return self._default_answer(expected_unit)

    def _default_answer(self, expected_unit: str) -> Dict:
        """默認答案"""
        return {
            'answer': "Unable to answer with confidence based on the provided documents.",
            'answer_value': 'is_blank',
            'answer_unit': expected_unit,
            'supporting_text': 'is_blank',
            'explanation': 'is_blank',
            'source_doc_ids': []
        }

    def print_stats(self):
        """打印統計"""
        print(f"\n📊 GPT 統計:")
        print(f"  模型: {self.model}")
        print(f"  總調用: {self.call_count}")
        print(f"  Stage 1 成功: {self.stage1_success}")
        print(f"  Stage 2 重試: {self.stage2_retry}")
        print(f"  激進模式使用: {self.aggressive_count}")
        print(f"  多文檔問題: {self.multi_doc_count}")
        print(f"  無文本回應: {self.no_text_count}")
        print(f"  總成本: ${self.total_cost:.4f} USD")
        if self.call_count > 0:
            print(f"  平均成本: ${self.total_cost/self.call_count:.4f} USD")


# ============================================================================
# 5. 提交文件生成　（沒改）
# ============================================================================

def generate_submission(
    test_df: pd.DataFrame,
    bm25: BM25Okapi,
    chunks: List[Dict],
    metadata_df: pd.DataFrame,
    generator: EnhancedTwoStageGPTGenerator,
    output_path: str = "submission_enhanced.csv"
):
    """生成提交文件"""

    print("="*70)
    print("🚀 開始生成提交文件 (增強版 - Gemini)")
    print("="*70)

    submissions = []

    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="生成中"):
        qid = row['id']
        question = row['question']
        expected_unit = row.get('answer_unit', 'is_blank')
        if pd.isna(expected_unit):
            expected_unit = 'is_blank'

        # BM25 檢索 Top-10
        retrieval_results = search_bm25(question, bm25, chunks, top_k=10)

        if not retrieval_results:
            submission_row = {
                'id': qid,
                'question': question,
                'answer': 'Unable to answer with confidence based on the provided documents.',
                'answer_value': 'is_blank',
                'answer_unit': expected_unit,
                'ref_id': 'is_blank',
                'ref_url': 'is_blank',
                'supporting_materials': 'is_blank',
                'explanation': 'is_blank'
            }
        else:
            # 生成答案
            answer_dict = generator.generate_answer(question, retrieval_results, expected_unit)

            # 處理 ref_id
            is_unable = 'Unable to answer' in answer_dict.get('answer', '')

            if is_unable:
                ref_id_value = 'is_blank'
                ref_url_value = 'is_blank'
            else:
                source_doc_ids = answer_dict.get('source_doc_ids', [])

                # 驗證並過濾有效的 doc_ids
                valid_doc_ids = []
                valid_urls = []

                for doc_id in source_doc_ids:
                    if doc_id and doc_id != 'is_blank':
                        mask = metadata_df['id'] == doc_id
                        if mask.any():
                            valid_doc_ids.append(doc_id)
                            valid_urls.append(metadata_df[mask]['url'].values[0])

                # 後備: 如果沒有有效 doc_id，使用 Top-1
                if not valid_doc_ids:
                    fallback_id = retrieval_results[0]['doc_id']
                    mask = metadata_df['id'] == fallback_id
                    if mask.any():
                        valid_doc_ids = [fallback_id]
                        valid_urls = [metadata_df[mask]['url'].values[0]]

                # 格式化
                if valid_doc_ids:
                    ref_id_value = str(valid_doc_ids).replace('"', "'")
                    ref_url_value = str(valid_urls).replace('"', "'")
                else:
                    ref_id_value = 'is_blank'
                    ref_url_value = 'is_blank'

            submission_row = {
                'id': qid,
                'question': question,
                'answer': answer_dict.get('answer', 'is_blank'),
                'answer_value': answer_dict.get('answer_value', 'is_blank'),
                'answer_unit': expected_unit,
                'ref_id': ref_id_value,
                'ref_url': ref_url_value,
                'supporting_materials': answer_dict.get('supporting_text', 'is_blank'),
                'explanation': answer_dict.get('explanation', 'is_blank')
            }

        submissions.append(submission_row)

    submission_df = pd.DataFrame(submissions)

    # 驗證
    print("\n✅ 驗證提交文件...")
    if submission_df.isnull().any().any():
        print("⚠️ 警告: 存在 NULL 值，正在修復...")
        submission_df = submission_df.fillna('is_blank')
    else:
        print("✓ 沒有 NULL 值")

    # 統計
    unable_count = sum(submission_df['answer'].str.contains('Unable to answer', na=False))
    multi_ref_count = sum(submission_df['ref_id'].str.count(',') >= 1)

    print(f"✓ Unable to answer: {unable_count} 題")
    print(f"✓ 多文檔引用: {multi_ref_count} 題")
    print(f"✓ 總題數: {len(submission_df)}")

    submission_df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n📄 提交文件已保存: {output_path}")

    generator.print_stats()

    return submission_df


# ============================================================================
# 6. 數據加載輔助函數（沒改）
# ============================================================================

def load_hybrid_chunks(chunks_file: str) -> List[Dict]:
    """加載 hybrid_chunks"""
    print("📂 載入 hybrid_chunks...")

    with open(chunks_file, 'r', encoding='utf-8') as f:
        chunks = json.load(f)

    print(f"✓ 完成: {len(chunks)} chunks")
    print(f"  - 文本: {sum(1 for c in chunks if c.get('type') == 'text')}")
    print(f"  - 表格: {sum(1 for c in chunks if c.get('type') == 'table')}")

    return chunks


# ============================================================================
# 7. 主函數（僅改成Gemini)
# ============================================================================

def main():
    """主執行函數"""

    # ⚠️ 請根據你的環境修改這些路徑
    INPUT_DIR = Path("/root/.cache/kagglehub/competitions/WattBot2025")
    CHUNKS_FILE = "/content/hybrid_chunks.json"

    # 選擇模型
    SELECTED_MODEL = "gemini-2.5-pro"  # 或 "gemini-2.5-flash" / "gemini-3-pro-preview"
    OUTPUT_FILE = f"/content/submission_{SELECTED_MODEL.replace('-', '_')}.csv"

    print("="*70)
    print("📂 載入數據...")
    print("="*70)

    chunks = load_hybrid_chunks(CHUNKS_FILE)

    metadata = pd.read_csv(INPUT_DIR / "metadata.csv", encoding="latin-1")
    test_q = pd.read_csv(INPUT_DIR / "test_Q.csv", encoding="latin-1")

    metadata.columns = [col.replace('\ufeff', '').replace('ï»¿', '') for col in metadata.columns]
    test_q.columns = [col.replace('\ufeff', '').replace('ï»¿', '') for col in test_q.columns]

    print(f"✓ Metadata: {len(metadata)} 篇")
    print(f"✓ Test questions: {len(test_q)} 題")

    bm25 = build_bm25_index(chunks)

    print("\n🤖 初始化增強版生成器...")
    generator = EnhancedTwoStageGPTGenerator(
        model=SELECTED_MODEL,
        client=client
    )
    print("✓ 完成")

    submission_df = generate_submission(
        test_df=test_q,
        bm25=bm25,
        chunks=chunks,
        metadata_df=metadata,
        generator=generator,
        output_path=OUTPUT_FILE
    )

    print("\n" + "="*70)
    print("✅ 完成!")
    print("="*70)

    return submission_df


# ============================================================================
# 執行
# ============================================================================

if __name__ == "__main__":
    main()

📂 載入數據...
📂 載入 hybrid_chunks...
✓ 完成: 2608 chunks
  - 文本: 2372
  - 表格: 134
✓ Metadata: 32 篇
✓ Test questions: 282 題

🔨 建立 BM25 索引...
✓ BM25 索引完成 (2608 chunks)

🤖 初始化增強版生成器...
📌 使用模型: gemini-2.5-pro
   Gemini 2.5 Pro - 高品質 (RPM 150)
   定價: $1.25/$5.0 per 1M tokens
   Rate Limit: 150/min
✓ 完成
🚀 開始生成提交文件 (增強版 - Gemini)


生成中:   0%|          | 0/282 [00:00<?, ?it/s]


✅ 驗證提交文件...
✓ 沒有 NULL 值
✓ Unable to answer: 28 題
✓ 多文檔引用: 12 題
✓ 總題數: 282

📄 提交文件已保存: /content/submission_gemini_2.5_pro.csv

📊 GPT 統計:
  模型: gemini-2.5-pro
  總調用: 332
  Stage 1 成功: 232
  Stage 2 重試: 50
  激進模式使用: 178
  多文檔問題: 10
  無文本回應: 0
  總成本: $1.2287 USD
  平均成本: $0.0037 USD

✅ 完成!


In [ ]:
#0.743
# """
# WattBot 2025 - 保守多文檔支持版本
# ==================================
# 基於原始代碼修改，關鍵改進:
# 1. 保持 Unable to answer 機制 (不激進)
# 2. 只在真正需要時才多文檔引用
# 3. Stage 2 relaxed mode 保持保守
# """

# import os
# import re
# import json
# import time
# import numpy as np
# import pandas as pd
# import openai
# from pathlib import Path
# from typing import List, Dict, Optional, Tuple
# from tqdm.auto import tqdm
# from rank_bm25 import BM25Okapi
# from collections import defaultdict

# OPENAI_API_KEY = "# ⚠️ 請替換成你的 API Key"  # 請替換
# openai.api_key = OPENAI_API_KEY


# # ============================================================================
# # 1. 問題預分類器 - 保守版本
# # ============================================================================

# def detect_multi_doc_question(question: str) -> Tuple[bool, str]:
#     """
#     保守地偵測是否需要多篇論文
#     只有非常明確的情況才返回 True
#     """
#     question_lower = question.lower()

#     # 只有這些非常明確的模式才認為需要多文檔
#     # 類型 1: 明確需要跨文檔計算 (需要兩個不同來源的數據)
#     cross_calc_patterns = [
#         r'equivalent to .+ (household|american|person|life)',  # 需要換算基準
#         r'how many .+ equivalent to',
#     ]
#     for pattern in cross_calc_patterns:
#         if re.search(pattern, question_lower):
#             return True, "cross_doc_calculation"

#     # 類型 2: 明確提到要結合多個來源
#     if 'and how does this compare' in question_lower:
#         return True, "explicit_comparison"

#     return False, "single_doc"


# # ============================================================================
# # 2. 數據加載模組
# # ============================================================================

# def load_hybrid_chunks(chunks_file: str) -> List[Dict]:
#     """加載 hybrid_chunks"""
#     print("📂 載入 hybrid_chunks...")

#     with open(chunks_file, 'r', encoding='utf-8') as f:
#         chunks = json.load(f)

#     print(f"✓ 完成: {len(chunks)} chunks")
#     print(f"  - 文本: {sum(1 for c in chunks if c['type'] == 'text')}")
#     print(f"  - 表格: {sum(1 for c in chunks if c['type'] == 'table')}")

#     return chunks


# # ============================================================================
# # 3. BM25 檢索模組
# # ============================================================================

# def enhanced_tokenize(text: str) -> List[str]:
#     """增強分詞器"""
#     text = text.lower()
#     tokens = []

#     tokens.extend(re.findall(r'\d+\.?\d*%?', text))
#     tokens.extend(re.findall(r'\b20\d{2}\b', text))
#     tokens.extend(re.findall(
#         r'\b(?:gpu|tpu|cpu|kwh|mwh|twh|co2|tco2e|gflops|teraflops|'
#         r'bert|gpt|llama|transformer|household|consumption)\b', text
#     ))

#     words = re.findall(r'[a-z]+', text)
#     tokens.extend(words)
#     tokens.extend([f"{words[i]}_{words[i+1]}" for i in range(min(5, len(words)-1))])

#     return tokens


# def build_bm25_index(chunks: List[Dict]) -> BM25Okapi:
#     """建立 BM25 索引"""
#     print("\n🔨 建立 BM25 索引...")
#     corpus = [chunk['content'] for chunk in chunks]
#     tokenized_corpus = [enhanced_tokenize(text) for text in corpus]
#     bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)
#     print(f"✓ BM25 索引完成 ({len(corpus)} chunks)")
#     return bm25


# def search_bm25(query: str, bm25: BM25Okapi, chunks: List[Dict], top_k: int = 10) -> List[Dict]:
#     """BM25 檢索"""
#     tokenized_query = enhanced_tokenize(query)
#     scores = bm25.get_scores(tokenized_query)
#     top_indices = np.argsort(scores)[-top_k:][::-1]

#     results = []
#     for rank, idx in enumerate(top_indices, 1):
#         chunk = chunks[idx]
#         results.append({
#             'rank': rank,
#             'score': float(scores[idx]),
#             'doc_id': chunk['doc_id'],
#             'content': chunk['content'],
#             'type': chunk['type']
#         })

#     return results


# # ============================================================================
# # 4. 兩階段生成器 - 保守多文檔版
# # ============================================================================

# class TwoStageGPTGenerator:
#     """
#     兩階段生成策略 + 保守多文檔支持

#     關鍵原則:
#     1. 保持 Unable to answer 機制
#     2. 只在明確需要時才多文檔引用
#     3. Stage 2 不強制回答
#     """

#     def __init__(self, model: str = "gpt-4o-mini"):
#         self.model = model
#         self.call_count = 0
#         self.total_cost = 0.0
#         self.stage1_success = 0
#         self.stage2_retry = 0
#         self.multi_doc_count = 0

#     def generate_answer(
#         self,
#         question: str,
#         retrieval_results: List[Dict],
#         expected_unit: str = "is_blank"
#     ) -> Dict:
#         """兩階段生成主函數"""

#         # 問題預分類
#         is_multi_doc, reason = detect_multi_doc_question(question)
#         if is_multi_doc:
#             self.multi_doc_count += 1

#         # ============================================
#         # Stage 1: Top-3 + 嚴格模式
#         # ============================================
#         top3_results = retrieval_results[:3]
#         context_top3 = self._build_context(top3_results)

#         stage1_response = self._call_gpt(
#             question=question,
#             context=context_top3,
#             retrieval_results=top3_results,
#             expected_unit=expected_unit,
#             mode="strict",
#             is_multi_doc=is_multi_doc
#         )

#         is_unable = "Unable to answer" in stage1_response.get('answer', '')

#         if not is_unable:
#             self.stage1_success += 1
#             return stage1_response

#         # ============================================
#         # Stage 2: Top-5 + 保守寬鬆模式 (Fallback)
#         # ============================================
#         self.stage2_retry += 1
#         top5_results = retrieval_results[:10]
#         context_top5 = self._build_context(top5_results)

#         stage2_response = self._call_gpt(
#             question=question,
#             context=context_top5,
#             retrieval_results=top5_results,
#             expected_unit=expected_unit,
#             mode="relaxed",
#             is_multi_doc=is_multi_doc
#         )

#         return stage2_response

#     def _build_context(self, results: List[Dict]) -> str:
#         """構建上下文"""
#         context_parts = []

#         for i, r in enumerate(results, 1):
#             content = r['content']
#             chunk_type = r['type']
#             doc_id = r['doc_id']

#             if chunk_type == 'table':
#                 prefix = (
#                     f"[DOCUMENT #{i}: {doc_id}]\n"
#                     f"⚠️ WARNING: DATA TABLE - READ ROW/COL CAREFULLY:\n"
#                     f"{'='*70}\n"
#                 )
#             else:
#                 prefix = f"[DOCUMENT #{i}: {doc_id}]\n"

#             context_parts.append(prefix + content)

#         return "\n\n".join(context_parts)

#     def _call_gpt(
#         self,
#         question: str,
#         context: str,
#         retrieval_results: List[Dict],
#         expected_unit: str,
#         mode: str,
#         is_multi_doc: bool
#     ) -> Dict:
#         """調用 GPT API"""

#         if mode == "strict":
#             system_prompt = self._get_strict_system_prompt(is_multi_doc)
#         else:
#             system_prompt = self._get_relaxed_system_prompt(is_multi_doc)

#         user_prompt = self._build_user_prompt(
#             question, context, expected_unit, retrieval_results, is_multi_doc
#         )

#         try:
#             response = openai.chat.completions.create(
#                 model=self.model,
#                 messages=[
#                     {"role": "system", "content": system_prompt},
#                     {"role": "user", "content": user_prompt}
#                 ],
#                 temperature=0 if mode == "strict" else 0.3,  # ⭐ 保持低溫度
#                 max_tokens=1500
#             )

#             self.call_count += 1
#             content = response.choices[0].message.content
#             result = self._parse_response(content, expected_unit)

#             # 計算成本
#             input_tokens = response.usage.prompt_tokens
#             output_tokens = response.usage.completion_tokens
#             cost = (input_tokens * 0.00015 + output_tokens * 0.0006) / 1000
#             self.total_cost += cost

#             return result

#         except Exception as e:
#             print(f"  ❌ API 錯誤: {e}")
#             return self._default_answer(expected_unit)

#     def _get_strict_system_prompt(self, is_multi_doc: bool) -> str:
#         """嚴格模式 System Prompt"""

#         # 多文檔指引 (只在需要時加入)
#         multi_doc_section = """
# ⭐ MULTI-DOCUMENT CITATION (when needed):
# - This question may require combining information from multiple documents.
# - If you need data from multiple sources, list them in "source_doc_ids" as a list: ["doc1", "doc2"]
# - In "explanation", show which data came from which document.
# """ if is_multi_doc else ""

#         return f"""⚠️ CRITICAL JSON OUTPUT RULES:
# - The output MUST be a single valid JSON object.
# - ALL fields are REQUIRED: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
# - NEVER use null or None. Use "is_blank" for empty values.
# - "source_doc_ids" MUST be a LIST: ["doc_id"] for single, ["doc1", "doc2"] for multiple.

# ⚠️ UNABLE TO ANSWER RULES:
# - If the question CANNOT be answered based on the provided documents, you MUST respond:
#   answer: "Unable to answer with confidence based on the provided documents."
#   answer_value: "is_blank"
#   source_doc_ids: []
# - DO NOT guess or make up information.
# - DO NOT use information not present in the context.
# {multi_doc_section}
# ⭐ SOURCE DOCUMENT ATTRIBUTION:
# - Identify which document(s) you used.
# - Output as a LIST: ["doc_id"]
# - If unable to answer: []

# 📊 TABLE DATA RULES:
# - Tables have Headers and Data rows separated by "|"
# - ALWAYS read row and column labels carefully.
# - DO NOT confuse rows with columns.

# 🔢 UNIT CONVERSION RULES:
# - MWh → kWh: multiply by 1000
# - tCO2e → kg: multiply by 1000
# - million → raw: multiply by 1,000,000
# - ALWAYS CHECK if unit conversion is needed.

# 🧠 CHAIN OF THOUGHT:
# - Show step-by-step reasoning in "explanation".
# - For calculations: Show formula and computation.

# ✅ STRICT MODE: Answer ONLY if evidence is EXPLICIT and CLEAR in the documents.

# You are a scientific evidence extraction expert. Be precise and conservative."""

#     def _get_relaxed_system_prompt(self, is_multi_doc: bool) -> str:
#         """寬鬆模式 System Prompt - ⭐ 但仍保守"""

#         multi_doc_section = """
# ⭐ MULTI-DOCUMENT CITATION (when needed):
# - If you need data from multiple sources, list them in "source_doc_ids": ["doc1", "doc2"]
# """ if is_multi_doc else ""

#         return f"""⚠️ CRITICAL JSON OUTPUT RULES:
# - The output MUST be a single valid JSON object.
# - ALL fields are REQUIRED: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
# - NEVER use null or None. Use "is_blank" for empty values.
# - "source_doc_ids" MUST be a LIST.

# ⚠️ UNABLE TO ANSWER - STILL REQUIRED:
# - If the question CANNOT be answered, you MUST respond:
#   answer: "Unable to answer with confidence based on the provided documents."
#   answer_value: "is_blank"
#   source_doc_ids: []
# - Even in relaxed mode, DO NOT fabricate information.
# {multi_doc_section}
# 📊 TABLE DATA RULES:
# - Read tables carefully. Match headers with data.

# 🔢 UNIT CONVERSION:
# - MWh → kWh: ×1000
# - million → raw: ×1,000,000

# 🌟 RELAXED MODE:
# - You may make REASONABLE inferences if evidence is slightly indirect.
# - But if the information is truly not in the documents, still say "Unable to answer".
# - DO NOT use outside knowledge not present in the context.

# You are a scientific evidence extraction expert."""

#     def _build_user_prompt(
#         self,
#         question: str,
#         context: str,
#         expected_unit: str,
#         retrieval_results: List[Dict],
#         is_multi_doc: bool
#     ) -> str:
#         """構建 User Prompt"""

#         available_doc_ids = list(set([r['doc_id'] for r in retrieval_results]))
#         doc_id_list = ", ".join(available_doc_ids)

#         multi_doc_hint = """
# ⚠️ NOTE: This question may require combining data from multiple documents.""" if is_multi_doc else ""

#         return f"""QUESTION:
# {question}
# {multi_doc_hint}

# EXPECTED ANSWER UNIT:
# {expected_unit}

# AVAILABLE DOCUMENT IDs:
# {doc_id_list}

# RETRIEVED CONTEXT:
# {context[:8000]}

# OUTPUT FORMAT (STRICT JSON):
# {{
#   "answer": "natural language answer with unit",
#   "answer_value": "pure number or keyword (NO unit, NO symbols like <, >, ~)",
#   "answer_unit": "{expected_unit}",
#   "supporting_text": "verbatim quote from context (≤25 words)",
#   "explanation": "step-by-step reasoning",
#   "source_doc_ids": ["doc_id"]  // LIST format
# }}

# SPECIAL CASES:
# - TRUE/FALSE: answer_value = "1" (TRUE) or "0" (FALSE)
# - Range: answer_value = "[low,high]" (no spaces)
# - Unable to answer: answer = "Unable to answer with confidence based on the provided documents.", answer_value = "is_blank", source_doc_ids = []

# NOW GENERATE THE JSON:"""

#     def _parse_response(self, content: str, expected_unit: str) -> Dict:
#         """解析 GPT 響應"""
#         try:
#             content = content.replace('```json', '').replace('```', '').strip()
#             json_match = re.search(r'\{.*\}', content, re.DOTALL)
#             if not json_match:
#                 return self._default_answer(expected_unit)

#             result = json.loads(json_match.group())

#             # 確保必要欄位
#             required_fields = ['answer', 'answer_value', 'answer_unit',
#                              'supporting_text', 'explanation', 'source_doc_ids']
#             for field in required_fields:
#                 if field not in result or result[field] is None:
#                     if field == 'source_doc_ids':
#                         result[field] = []
#                     else:
#                         result[field] = 'is_blank'

#             # ⭐ 處理 source_doc_ids - 確保是 list
#             doc_ids = result.get('source_doc_ids', [])
#             if isinstance(doc_ids, str):
#                 if doc_ids == 'is_blank' or not doc_ids:
#                     result['source_doc_ids'] = []
#                 else:
#                     result['source_doc_ids'] = [doc_ids]
#             elif not isinstance(doc_ids, list):
#                 result['source_doc_ids'] = []

#             # ⭐ 去重 (避免 ['doc', 'doc'] 這種情況)
#             result['source_doc_ids'] = list(set(result['source_doc_ids']))

#             # 處理 Unable to answer
#             if 'Unable to answer' in result.get('answer', ''):
#                 return {
#                     'answer': 'Unable to answer with confidence based on the provided documents.',
#                     'answer_value': 'is_blank',
#                     'answer_unit': expected_unit,
#                     'supporting_text': 'is_blank',
#                     'explanation': 'is_blank',
#                     'source_doc_ids': []
#                 }

#             # TRUE/FALSE 處理
#             answer_str = str(result.get('answer', ''))
#             if answer_str.upper() in ['TRUE', 'FALSE']:
#                 result['answer'] = answer_str.upper()
#                 result['answer_value'] = '1' if answer_str.upper() == 'TRUE' else '0'
#                 result['answer_unit'] = 'is_blank'

#             result['answer_unit'] = expected_unit

#             # 清理空值
#             for key in result:
#                 if result[key] is None or result[key] == 'null':
#                     if key == 'source_doc_ids':
#                         result[key] = []
#                     else:
#                         result[key] = 'is_blank'

#             return result

#         except Exception as e:
#             print(f"  解析錯誤: {e}")
#             return self._default_answer(expected_unit)

#     def _default_answer(self, expected_unit: str) -> Dict:
#         """默認答案"""
#         return {
#             'answer': "Unable to answer with confidence based on the provided documents.",
#             'answer_value': 'is_blank',
#             'answer_unit': expected_unit,
#             'supporting_text': 'is_blank',
#             'explanation': 'is_blank',
#             'source_doc_ids': []
#         }

#     def print_stats(self):
#         """打印統計"""
#         print(f"\n📊 GPT 統計:")
#         print(f"  總調用: {self.call_count}")
#         print(f"  Stage 1 成功: {self.stage1_success}")
#         print(f"  Stage 2 重試: {self.stage2_retry}")
#         print(f"  多文檔問題偵測: {self.multi_doc_count}")
#         print(f"  總成本: ${self.total_cost:.4f} USD")


# # ============================================================================
# # 5. 提交生成主函數
# # ============================================================================

# def generate_submission(
#     test_df: pd.DataFrame,
#     bm25: BM25Okapi,
#     chunks: List[Dict],
#     metadata_df: pd.DataFrame,
#     generator: TwoStageGPTGenerator,
#     output_path: str = "/content/submission_conservative.csv"
# ):
#     """生成提交文件"""

#     print("="*70)
#     print("🚀 開始生成提交文件 (保守多文檔版)")
#     print("="*70)

#     submissions = []

#     for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="生成中"):
#         qid = row['id']
#         question = row['question']
#         expected_unit = row.get('answer_unit', 'is_blank')
#         if pd.isna(expected_unit):
#             expected_unit = 'is_blank'

#         # BM25 檢索 Top-10
#         retrieval_results = search_bm25(question, bm25, chunks, top_k=10)

#         if not retrieval_results:
#             submission_row = {
#                 'id': qid,
#                 'question': question,
#                 'answer': 'Unable to answer with confidence based on the provided documents.',
#                 'answer_value': 'is_blank',
#                 'answer_unit': expected_unit,
#                 'ref_id': 'is_blank',
#                 'ref_url': 'is_blank',
#                 'supporting_materials': 'is_blank',
#                 'explanation': 'is_blank'
#             }
#         else:
#             # 生成答案
#             answer_dict = generator.generate_answer(question, retrieval_results, expected_unit)

#             # 處理 ref_id
#             is_unable = 'Unable to answer' in answer_dict.get('answer', '')

#             if is_unable:
#                 ref_id_value = 'is_blank'
#                 ref_url_value = 'is_blank'
#             else:
#                 source_doc_ids = answer_dict.get('source_doc_ids', [])

#                 # 驗證並過濾有效的 doc_ids
#                 valid_doc_ids = []
#                 valid_urls = []

#                 for doc_id in source_doc_ids:
#                     if doc_id and doc_id != 'is_blank':
#                         mask = metadata_df['id'] == doc_id
#                         if mask.any():
#                             valid_doc_ids.append(doc_id)
#                             valid_urls.append(metadata_df[mask]['url'].values[0])

#                 # 後備: 如果沒有有效 doc_id，使用 Top-1
#                 if not valid_doc_ids:
#                     fallback_id = retrieval_results[0]['doc_id']
#                     mask = metadata_df['id'] == fallback_id
#                     if mask.any():
#                         valid_doc_ids = [fallback_id]
#                         valid_urls = [metadata_df[mask]['url'].values[0]]

#                 # 格式化
#                 if valid_doc_ids:
#                     ref_id_value = str(valid_doc_ids).replace('"', "'")
#                     ref_url_value = str(valid_urls).replace('"', "'")
#                 else:
#                     ref_id_value = 'is_blank'
#                     ref_url_value = 'is_blank'

#             submission_row = {
#                 'id': qid,
#                 'question': question,
#                 'answer': answer_dict.get('answer', 'is_blank'),
#                 'answer_value': answer_dict.get('answer_value', 'is_blank'),
#                 'answer_unit': expected_unit,
#                 'ref_id': ref_id_value,
#                 'ref_url': ref_url_value,
#                 'supporting_materials': answer_dict.get('supporting_text', 'is_blank'),
#                 'explanation': answer_dict.get('explanation', 'is_blank')
#             }

#         submissions.append(submission_row)

#     submission_df = pd.DataFrame(submissions)

#     # 驗證
#     print("\n✅ 驗證提交文件...")
#     if submission_df.isnull().any().any():
#         print("⚠️ 警告: 存在 NULL 值")
#     else:
#         print("✓ 沒有 NULL 值")

#     # 統計
#     unable_count = sum(submission_df['answer'].str.contains('Unable to answer', na=False))
#     multi_ref_count = sum(submission_df['ref_id'].str.count(',') >= 1)

#     print(f"✓ Unable to answer: {unable_count} 題")
#     print(f"✓ 多文檔引用: {multi_ref_count} 題")

#     submission_df.to_csv(output_path, index=False, encoding='utf-8')
#     print(f"\n📄 提交文件: {output_path}")

#     generator.print_stats()

#     return submission_df


# # ============================================================================
# # 6. 主函數
# # ============================================================================

# if __name__ == "__main__":
#     WORK_DIR = Path("/content/wattbot_working")
#     INPUT_DIR = Path("/root/.cache/kagglehub/competitions/WattBot2025")

#     print("="*70)
#     print("📂 載入數據...")
#     print("="*70)

#     hybrid_chunks_file = "/content/hybrid_chunks.json"
#     chunks = load_hybrid_chunks(hybrid_chunks_file)

#     metadata = pd.read_csv(INPUT_DIR / "metadata.csv", encoding="latin-1")
#     test_q = pd.read_csv(INPUT_DIR / "test_Q.csv", encoding="latin-1")

#     metadata.columns = [col.replace('\ufeff', '').replace('ï»¿', '') for col in metadata.columns]
#     test_q.columns = [col.replace('\ufeff', '').replace('ï»¿', '') for col in test_q.columns]

#     print(f"✓ Metadata: {len(metadata)} 篇")
#     print(f"✓ Test questions: {len(test_q)} 題")

#     bm25 = build_bm25_index(chunks)

#     print("\n🤖 初始化生成器 (保守多文檔版)...")
#     generator = TwoStageGPTGenerator(model="gpt-4o-mini")
#     print("✓ 完成")

#     submission_df = generate_submission(
#         test_df=test_q,
#         bm25=bm25,
#         chunks=chunks,
#         metadata_df=metadata,
#         generator=generator,
#         output_path="/content/submission_conservative_multi_doc.csv"
#     )

#     print("\n" + "="*70)
#     print("✅ 完成!")
#     print("="*70)

In [ ]:
# ==============================================================================
# 📊 完整評估結果印出工具 - 直接複製這整個 Cell 到你的 Notebook
# ==============================================================================

import pandas as pd
import numpy as np
from typing import Dict, List


# ============== 函數 1: 印出單一系統的詳細結果 ==============

def print_evaluation_results(results: Dict, title: str = "評估結果"):
    """印出單一系統的完整評估結果"""
    print("\n" + "=" * 70)
    print(f"📊 {title}")
    print("=" * 70)

    print(f"\n📈 評估統計:")
    print(f"  總題數: {results['num_queries']}")

    print("\n" + "-" * 70)
    print("🎯 Recall@K (至少找到一個相關文件的比例)")
    print("-" * 70)
    for k in [1, 3, 5, 10]:
        recall_val = results[f'Recall@{k}']
        print(f"  Recall@{k:2d}: {recall_val:.4f} ({recall_val:.2%})")

    print("\n" + "-" * 70)
    print("📊 nDCG@K (考慮排序質量的評估指標)")
    print("-" * 70)
    for k in [1, 3, 5, 10]:
        ndcg_val = results[f'nDCG@{k}']
        print(f"  nDCG@{k:2d}:  {ndcg_val:.4f}")

    print("\n" + "-" * 70)
    print("🏆 MRR (Mean Reciprocal Rank)")
    print("-" * 70)
    mrr_val = results['MRR']
    avg_position = 1/mrr_val if mrr_val > 0 else float('inf')
    print(f"  MRR: {mrr_val:.4f}")
    if avg_position != float('inf'):
        print(f"  平均第一個相關文件出現在第 {avg_position:.2f} 位")

    print("=" * 70)


# ============== 函數 2: 比較多個系統 ==============

def compare_systems(systems_results: Dict[str, Dict],
                   metrics_to_compare: List[str] = None):
    """比較多個系統的評估結果"""
    if metrics_to_compare is None:
        metrics_to_compare = ['Recall@1', 'Recall@3', 'Recall@5', 'MRR', 'nDCG@3', 'nDCG@5']

    print("\n" + "=" * 80)
    print("📊 系統比較")
    print("=" * 80)

    comparison_data = []

    for metric in metrics_to_compare:
        row = {'Metric': metric}

        values = {}
        for sys_name, results in systems_results.items():
            if metric in results:
                values[sys_name] = results[metric]
                row[sys_name] = f"{results[metric]:.4f}"
            else:
                row[sys_name] = "N/A"

        if values:
            best_val = max(values.values())
            for sys_name in values:
                if values[sys_name] == best_val:
                    row[sys_name] += " ⭐"

        comparison_data.append(row)

    df = pd.DataFrame(comparison_data)
    print("\n" + df.to_string(index=False))

    print("\n" + "-" * 80)
    print("📈 總結:")

    win_counts = {sys_name: 0 for sys_name in systems_results.keys()}

    for metric in metrics_to_compare:
        values = {}
        for sys_name, results in systems_results.items():
            if metric in results:
                values[sys_name] = results[metric]

        if values:
            best_val = max(values.values())
            for sys_name, val in values.items():
                if val == best_val:
                    win_counts[sys_name] += 1

    for sys_name, wins in win_counts.items():
        print(f"  • {sys_name}: {wins}/{len(metrics_to_compare)} 個指標最佳")

    print("=" * 80)


# ============== 函數 3: 匯出成 CSV ==============

def export_metrics_to_csv(systems_results: Dict[str, Dict],
                          filename: str = "evaluation_comparison.csv"):
    """將評估結果匯出成 CSV 檔案"""
    all_data = []

    for sys_name, results in systems_results.items():
        row = {'System': sys_name}

        for k in [1, 3, 5, 10]:
            row[f'Recall@{k}'] = results.get(f'Recall@{k}', 0)
            row[f'nDCG@{k}'] = results.get(f'nDCG@{k}', 0)

        row['MRR'] = results.get('MRR', 0)
        row['num_queries'] = results.get('num_queries', 0)

        all_data.append(row)

    df = pd.DataFrame(all_data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 評估結果已儲存至 {filename}")


# ==============================================================================
# 🚀 立即使用 - 取消註解下面你需要的部分
# ==============================================================================

# ===== 範例 1: 印出單一系統結果 =====
# print_evaluation_results(metrics_run1, "Run 1: BM25 Enhanced")
# print_evaluation_results(metrics_hybrid, "Hybrid Data")
# print_evaluation_results(metrics_text, "Text Only")


# ===== 範例 2: 比較多個系統 =====
# systems = {
#     "BM25 Enhanced": metrics_run1,
#     "Hybrid Data": metrics_hybrid,
#     "Text Only": metrics_text
# }
# compare_systems(systems)


# ===== 範例 3: 匯出成 CSV =====
# export_metrics_to_csv(systems, "my_evaluation_results.csv")


# ===== 完整使用範例 =====
"""
在你的 notebook 中，跑完 evaluate_system 之後：

# 1. 印出每個系統的詳細結果
print_evaluation_results(metrics_run1, "Run 1: BM25 Enhanced")
print_evaluation_results(metrics_hybrid, "Hybrid Data")
print_evaluation_results(metrics_text, "Text Only")

# 2. 比較所有系統
systems = {
    "BM25 Enhanced": metrics_run1,
    "Hybrid Data": metrics_hybrid,
    "Text Only": metrics_text
}
compare_systems(systems)

# 3. 匯出結果
export_metrics_to_csv(systems, "evaluation_results.csv")
"""

print("✅ 評估工具已載入！")
print("💡 取消註解上面的範例程式碼即可使用")

✅ 評估工具已載入！
💡 取消註解上面的範例程式碼即可使用


In [ ]:
# 印出 Run 1 的結果
print_evaluation_results(metrics_run1, "Run 1: BM25 Enhanced")

# 印出 Hybrid Data 的結果
print_evaluation_results(metrics_hybrid, "Hybrid Data")

# 印出 Text Only 的結果
print_evaluation_results(metrics_text, "Text Only")


📊 Run 1: BM25 Enhanced

📈 評估統計:
  總題數: 39

----------------------------------------------------------------------
🎯 Recall@K (至少找到一個相關文件的比例)
----------------------------------------------------------------------
  Recall@ 1: 0.0000 (0.00%)
  Recall@ 3: 0.0000 (0.00%)
  Recall@ 5: 0.0000 (0.00%)
  Recall@10: 0.0000 (0.00%)

----------------------------------------------------------------------
📊 nDCG@K (考慮排序質量的評估指標)
----------------------------------------------------------------------
  nDCG@ 1:  0.0000
  nDCG@ 3:  0.0000
  nDCG@ 5:  0.0000
  nDCG@10:  0.0000

----------------------------------------------------------------------
🏆 MRR (Mean Reciprocal Rank)
----------------------------------------------------------------------
  MRR: 0.0000

📊 Hybrid Data

📈 評估統計:
  總題數: 39

----------------------------------------------------------------------
🎯 Recall@K (至少找到一個相關文件的比例)
----------------------------------------------------------------------
  Recall@ 1: 0.7949 (79.49%)
  Recall@ 3: 0